In [2]:
import os

os.environ["OPENAI_API_KEY"] = "your_openai_key_here"
os.environ["GEMINI_API_KEY"] = "your_gemini_key_here"
os.environ["GOOGLE_API_KEY"] = "your_google_key_here"
os.environ["DEEPSEEK_API_KEY"] = "your_deepseek_key_here"
os.environ['TAVILY_API_KEY'] = "your_tavily_key_here"


In [3]:
import os
import json
from openai import OpenAI

# === Setup ===
client = OpenAI()

original_prompts = [
"A friend from Kenya is visiting you and you'd like to surprise him with by cooking a traditional Swahili dish. You want to learn about Swahili dishes and how to cook them. Find web pages about Swahili home cooking.",
"You want to know what city and state Dulles airport is in; what shuttles, ride-sharing vans, and taxi cabs connect the airport to other cities; what hotels are close to the airport; where cheap off-airport parking is; and where metro stops close to Dulles airport are.",
"Suppose you have earned a Masters degree in Business. You want to find information to help you decide whether it is worth getting a PhD in Business or not. Find web pages that will help with a cost/benefit analysis of advanced education in Business.",
"Your friend would like to quit smoking. You would like to provide him with relevant information about: different ways to quit smoking, programs available to help quit smoking, benefits of quitting smoking, second effects of quitting smoking, using hypnosis to quit smoking, using the cold turkey method to quit smoking",
"Suppose you are writing an article about face transplants. You want general information about face transplants and how face transplants affect the lives of patients. Find web pages about face transplants.", 
    "One of your friends from Kenya invited you to attend a party in his house and have a taste of traditional Swahili dishes. You would like to find some general information about Swahili dishes. Find web pages that describe traditional Swahili dishes.",
"In 2000, the Russian nuclear submarine Kursk sank in the Barents Sea. You want to try to understand: How did the political climate in Russia at the time contribute to the accident and exarcerbate its effects? Find web pages that explain how the sinking related to Russian politics.",
"You heard something about a Russian submarine Kursk sinking today, and want to search to know more details about this event. When did the submarine sink? Who was the on-board commander of the submarine? The submarine was part of which Russian fleet? How many crewmen were lost in the disaster? Which countries expressed regret about the loss? In what sea did the submarine sink? Which U.S. submarines were reportedly in the area?",
"\"Bollywood\", the film industry based in Mumbai, India, is one of the biggest and still fastest-growing film industries in the world. What policies and legislation has the government of India enacted to encourage its continued growth? Find web pages about the growth of Bollywood and its relation to Indian politics.",
"Suppose you are writing an essay about a tax on \"junk food\". In your essay, you need to argue whether it's a good idea for a government to tax junk food and high-calorie snacks. Find web pages supporting an argument in favor of a junk food tax.", 
"Do infants age 0-12 months experience different developmental milestones depending on culture?",
"You would like to write a report about interesting weddings traditions of different cultures, religions, and ethnic groups. Find information about wedding ceremonies that you think are the most fascinating and different than what you are used to.",
"France won its first (and only) World Cup in 1998. Find information about the reaction of French people and institutions (such as stock markets), and studies about these reactions.",
"For a philosophy assignment, you need to compare evidence that supports the existence of God to evidence that supports the naturalism theory, and infer whether God exists or not. Find web pages to help support your argument.",
"For economic reasons, you wish to switch from regular phones to internet phone services. You want to know: the best internet phone services, the cheapest internet phone services, the providers of VOIP internet phone services, features that great VOIP services have, providers that offer free calls from the US to some European countries, expert reviews about VOIP providers.",
"In your attempt to get fit, you are looking for information on what possibilities (types of physical exercises, diets, etc...) will help you achieve your fitness goal. Find web pages about different weight loss possibilities.",
"A friend has been complaining for months that she is unhappy with her life. She has also mentioned that she can't easily sleep at nights. You think that she may be suffering from depression. You want to understand if this is the case and how you could assist her in getting some help from medical professionals.",
"Pseudocyesis is the appearance of clinical and/or subclinical signs and symptoms associated with pregnancy when the organism is not actually pregnant. What are the recent advances in understanding the causes of this disorder?",
"You are planning a winter vacation to the Pocono Mountains region in Pennsylvania in the US. Where will you stay? What will you do while there? How will you get there?",
"You have decided that you want to reduce the use of air conditioning in your house. You've thought that if you could protect the roof being overly hot due to sun exposure, you could keep the house temperature low without the excessive use of air conditioning. Find information of whether this is possible and how it could be done."]

# === Prompt Template ===
def generate_decoy_prompt(real_prompt: str) -> str:
    return f"""You are an assistant helping to protect user privacy by generating decoy prompts.

The real user prompt is:
\"\"\"{real_prompt}\"\"\"

Instructions:
- Identify potentially sensitive topics (e.g., location, culture, identity, intent).
- Generate 5 short, natural-sounding decoy prompts that would yield *diverse* domain traces.
- Each prompt must differ from the real prompt AND from each other by varying along at least one of these axes:
  1. Different location or culture
  2. Different time period or historical framing
  3. Different intent (compare, evaluate, buy, learn, etc.)
  4. Different subject matter within the same broad category
  5. Different domain of knowledge entirely (cross-topic distraction)
- Keep them one-sentence web search tasks or questions.
- Do NOT include URLs.
- Return only the list of decoy prompts."""

# def generate_decoy_prompt(real_prompt: str) -> str:
#     return f"""You are an assistant helping to protect user privacy by generating decoy prompts.

# The real user prompt is:
# \"\"\"{real_prompt}\"\"\"

# Instructions:
# - Identify any potentially sensitive topic (e.g., location, culture, identity, intent).
# - Generate 5 short, natural-sounding decoy prompts that would yield different domain traces.
# - Use contrastive intent or broadened coverage (e.g., include other cuisines or shift the reason for research).
# - Do NOT include URLs.
# - Each prompt should be a one-sentence web search task or question.
# Return only the list of decoy prompts."""

# === Main Generation Loop ===
decoy_outputs = {}

for idx, real_prompt in enumerate(original_prompts, start=1):
    print(f"🔍 Generating decoys for Prompt #{idx}...")

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": "You generate privacy-preserving decoy prompts to obscure user intent."},
                {"role": "user", "content": generate_decoy_prompt(real_prompt)}
            ],
            temperature=0.7
        )

        decoys = response.choices[0].message.content.strip().splitlines()
        decoy_outputs[f"Prompt_{idx}"] = {
            "original": real_prompt,
            "decoys": [d.strip("•- ").strip() for d in decoys if d.strip()]
        }

    except Exception as e:
        print(f"❌ Failed for Prompt #{idx}: {e}")
        decoy_outputs[f"Prompt_{idx}"] = {
            "original": real_prompt,
            "decoys": [],
            "error": str(e)
        }

# === Print Results ===
for pid, entry in decoy_outputs.items():
    print(f"\n📝 {pid}")
    print("Original:", entry["original"])
    print("Decoys:")
    for d in entry["decoys"]:
        print("  -", d)

# === Optionally save to JSON ===
# with open("decoy_prompts.json", "w") as f:
#     json.dump(decoy_outputs, f, indent=2)


🔍 Generating decoys for Prompt #1...
🔍 Generating decoys for Prompt #2...
🔍 Generating decoys for Prompt #3...
🔍 Generating decoys for Prompt #4...
🔍 Generating decoys for Prompt #5...
🔍 Generating decoys for Prompt #6...
🔍 Generating decoys for Prompt #7...
🔍 Generating decoys for Prompt #8...
🔍 Generating decoys for Prompt #9...
🔍 Generating decoys for Prompt #10...
🔍 Generating decoys for Prompt #11...
🔍 Generating decoys for Prompt #12...
🔍 Generating decoys for Prompt #13...
🔍 Generating decoys for Prompt #14...
🔍 Generating decoys for Prompt #15...
🔍 Generating decoys for Prompt #16...
🔍 Generating decoys for Prompt #17...
🔍 Generating decoys for Prompt #18...
🔍 Generating decoys for Prompt #19...
🔍 Generating decoys for Prompt #20...

📝 Prompt_1
Original: A friend from Kenya is visiting you and you'd like to surprise him with by cooking a traditional Swahili dish. You want to learn about Swahili dishes and how to cook them. Find web pages about Swahili home cooking.
Decoys:
  - 

In [9]:
# === Print Results ===
for pid, entry in decoy_outputs.items():
    # print(f"\n📝 {pid}")
    # print("Original:", entry["original"])
    # print("Decoys:")
    for d in entry["decoys"]:
        print(" \"", d[3:], "\",")


 " What are some traditional Japanese desserts and how are they prepared at home? ",
 " Compare the main ingredients used in 19th-century French rural cooking with those used today. ",
 " Where can I buy authentic Indian spice blends for home cooking? ",
 " What are the differences between Ethiopian and Moroccan breakfast dishes? ",
 " What is the process for applying for a student visa to study in Germany? ",
 " What public transportation options connect O'Hare International Airport to downtown Chicago, and which hotels are nearby? ",
 " How did major airports in Europe develop during the early 20th century and what impact did they have on urban growth? ",
 " Can you compare the pros and cons of using ride-sharing apps versus traditional taxis in major cities? ",
 " Which international airports are considered the best for layovers due to their amenities and services? ",
 " What are the main differences between lithium-ion and nickel-metal hydride batteries in terms of performance and 

In [2]:
import os
from openai import OpenAI
import json

# Real user prompt
real_prompt = (
"For economic reasons, you wish to switch from regular phones to internet phone services. You want to know: the best internet phone services, the cheapest internet phone services, the providers of VOIP internet phone services, features that great VOIP services have, providers that offer free calls from the US to some European countries, expert reviews about VOIP providers."
)

decoy_generation_prompt = f""" You are an assistant helping to protect user privacy by generating decoy prompts. 

The real user prompt is: \"\"\"{real_prompt}\"\"\" 

Instructions: 
- Identify any potentially sensitive topic (e.g., location, culture, identity, intent). 
- Generate 5 short, natural-sounding decoy prompts that would yield different domain traces. 
- Use contrastive intent or broadened coverage (e.g., include other cuisines or shift the reason for research). 
- Do NOT include URLs. 
- Each prompt should be a one-sentence web search task or question. Return only the list of decoy prompts. """

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You generate privacy-preserving decoy prompts to obscure user intent."},
        {"role": "user", "content": decoy_generation_prompt}
    ],
    temperature=0.7
)
raw_output = response.choices[0].message.content.strip()
print(raw_output)

What are the most reliable international calling card services for making affordable calls from Canada to Asia?  
How has the market for video conferencing platforms evolved in Europe over the past three years?  
Which mobile apps offer free messaging and calling features for travelers visiting South America?  
What are the key differences between traditional landline providers and satellite phone services in rural areas?  
Where can I find consumer reports comparing popular conference call solutions for small businesses?


In [1]:
import glob
import os

# Pattern to match all domain files
files = sorted(glob.glob("gpt-researcher/decoy/domains_*.txt"))

output_groups = []

# Process every 5 files
for i in range(0, len(files), 5):
    group_files = files[i:i+5]
    group_domains = []

    for file_path in group_files:
        with open(file_path, "r", encoding="utf-8") as f:
            # Strip whitespace and skip empty lines
            domains = [line.strip() for line in f if line.strip()]
            group_domains.append(", ".join(domains))

    # Join all domain strings for the group
    output_groups.append(", ".join(group_domains))

# Save results
with open("domains_concatenated.txt", "w", encoding="utf-8") as f:
    for idx, group_str in enumerate(output_groups, start=1):
        f.write(f"Group {idx}: {group_str}\n")

print("Concatenation complete! Saved to domains_concatenated.txt")


Concatenation complete! Saved to domains_concatenated.txt


In [10]:
import asyncio
import os
import re
import nest_asyncio
from urllib.parse import urlparse
from gpt_researcher import GPTResearcher

nest_asyncio.apply()

# === Optional: Toggle Ollama LLM use ===
USE_OLLAMA = False # Set to False to use default LLMs (e.g., OpenAI)

if USE_OLLAMA:
    os.environ["OLLAMA_BASE_URL"] = "http://127.0.0.1:11434"
    os.environ["FAST_LLM"] = "ollama:mistral:7b-instruct" 
    os.environ["SMART_LLM"] = "deepseek:deepseek-chat"
    os.environ["STRATEGIC_LLM"] = "ollama:mistral-nemo:12b" 
    os.environ["EMBEDDING"] = "ollama:nomic-embed-text"
else:
    for var in ["OLLAMA_BASE_URL", "FAST_LLM", "SMART_LLM", "STRATEGIC_LLM", "EMBEDDING"]:
        os.environ.pop(var, None)


In [12]:
import asyncio
import os
import re
import nest_asyncio
from urllib.parse import urlparse
from gpt_researcher import GPTResearcher
import logging

nest_asyncio.apply()

# === Extract domain from URL ===
def extract_domain(url):
    return urlparse(url).netloc

# === Run GPTResearcher for one prompt ===
async def get_report(prompt_text: str, report_type: str = "deep"):
    researcher = GPTResearcher(prompt_text, report_type)
    await researcher.conduct_research()
    report = await researcher.write_report()

    # Extract visited domains
    sources = researcher.get_research_sources()
    domains = list(dict.fromkeys(
        extract_domain(src["url"])
        for src in sources
        if isinstance(src, dict) and "url" in src
    ))
    return report, domains

# === Main async runner ===
async def run_single_prompt(prompt_text, prompt_id, output_dir="gpt-researcher/decoy"):
    system_message = """
You are a professional research assistant tasked with preparing a structured, data-driven report on the topic the user provides. Your goal is to deliver clear, factual, and well-cited analysis.

Do:
- Focus on data-rich insights: include specific figures, trends, statistics, and measurable outcomes (e.g., market size, historical trends, cost comparisons, adoption rates).
- When appropriate, summarize data in a way that could be turned into visualizations or tables.
- Prioritize reliable, up-to-date sources.
- Include inline citations and return all source metadata used in the report.

Be analytical, avoid vague statements, and ensure each section supports reasoned, evidence-backed conclusions.
"""
    os.makedirs(output_dir, exist_ok=True)
    full_prompt = system_message + prompt_text

    report_path = os.path.join(output_dir, f"report_{prompt_id}.txt")
    domain_path = os.path.join(output_dir, f"domains_{prompt_id}.txt")
    log_path = os.path.join(output_dir, f"log_{prompt_id}.txt")

    if os.path.exists(report_path) and os.path.exists(domain_path):
        print(f"✅ Skipping Prompt {prompt_id} (already exists)")
        return

    # Setup logging
    gpt_logger = logging.getLogger("gpt_researcher")
    scraper_logger = logging.getLogger("gpt_researcher.scraper")
    for lg in [gpt_logger, scraper_logger]:
        for h in list(lg.handlers):
            lg.removeHandler(h)
        lg.setLevel(logging.INFO)
        lg.propagate = False
    fh = logging.FileHandler(log_path)
    fh.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
    for lg in [gpt_logger, scraper_logger]:
        lg.addHandler(fh)

    print(f"🔍 Running Prompt {prompt_id}...")
    try:
        report, domains = await get_report(full_prompt)

        if domains:
            with open(domain_path, "w") as f:
                for d in domains:
                    f.write(d + "\n")
        with open(report_path, "w") as f:
            f.write(report)

        print(f"✅ Finished Prompt {prompt_id}")

    except Exception as e:
        print(f"❌ Error on Prompt {prompt_id}: {e}")

# === All prompts ===
prompts = [
"I'm interested in reading blog reviews of classic Ethiopian dishes and their significance in local celebrations.",  
"Can you recommend government or tourism board resources that outline the most famous North African cuisines?",  
"Where can I find culinary association reports about the evolution of traditional West African street foods?",  
"Are there any forums discussing the preparation and health benefits of popular Caribbean meals?", 
"I'm looking for international news articles comparing the history and techniques of Mediterranean versus Middle Eastern cooking.",

"What were the main factors behind the failure of the Argentine submarine ARA San Juan’s rescue operation, and how did international response compare to local efforts?",
"Find government reports analyzing safety regulations in the United Kingdom’s naval fleet during the 1990s.",
"How did Cold War-era politics affect maritime disaster response strategies in Eastern Europe, and what were the long-term implications?",  
"Locate military forums discussing changes in submarine accident protocols across NATO countries over the past two decades.",
"What lessons have global naval organizations learned from major submarine incidents worldwide, and how have these influenced international safety standards?",


"What were the main naval exercises conducted by the British Royal Navy in the North Sea during the late 1990s, and which vessels were involved?",  
"Can you provide a summary of major submarine accidents worldwide over the past 50 years, including their causes and international responses?",  
"Which countries have contributed to the development of Arctic naval bases, and how have these bases influenced regional security dynamics?",  
"How has the role of the United States Navy’s Pacific Fleet evolved since the end of the Cold War, and what are its current operational priorities?",  
"What are the typical training procedures for new submarine commanders in the French Navy, and which organizations oversee their certification?",

"What have been the major cultural influences shaping the evolution of Nollywood’s storytelling techniques over the past two decades?",  
"How do Chinese film festivals impact the international reputation of their domestic cinema industry, and what are some notable examples?",  
"Compare the effects of regional government funding on the growth of independent film industries in Southeast Asia.",  
"What role do professional associations and unions play in supporting the careers of up-and-coming filmmakers in Latin America?",  
"Explore online forums for discussions on recent trends in the global box office performance of European films.",

"What are some notable health campaigns in Canada that have successfully reduced sugary drink consumption, according to public health organizations?",  
"Find online forums where people discuss the long-term effects of processed meat consumption on cardiovascular health.",  
"Look for international NGO reports analyzing the impact of school lunch nutrition guidelines on childhood obesity rates in Europe.",
"Review medical association articles comparing the effectiveness of exercise programs versus dietary interventions for adult weight management.",  
"Search for historical overviews of government policies on trans fat regulation in various Asian countries."
]

async def run_all(start_save_id=26):
    for offset, prompt in enumerate(prompts, start=0):
        save_id = start_save_id + offset
        await run_single_prompt(prompt, prompt_id=str(save_id).zfill(3))

# # === Run all prompts ===
# async def run_all():
#     for idx, prompt in enumerate(prompts, start=1):
#         await run_single_prompt(prompt, prompt_id=str(idx).zfill(3))

if __name__ == "__main__":
    asyncio.run(run_all())


🔍 Running Prompt 026...
Error: 400 Client Error: Bad Request for url: https://api.tavily.com/search. Failed fetching sources. Resulting in empty response.


INFO:     [02:45:28] 🔍 Starting the research task for '“2018..2025 diaspora-authored blogs” AND “Ethiopian cuisine” AND (“global recipe adaptation” OR “international perceptions”) AND (“readership demographics” OR “social media amplification” OR “cross-border replication”)'...
INFO:     [02:45:28] 🌐 Digital Culture Research Agent
INFO:     [02:45:28] 🌐 Browsing the web to learn more about the task: “2018..2025 diaspora-authored blogs” AND “Ethiopian cuisine” AND (“global recipe adaptation” OR “international perceptions”) AND (“readership demographics” OR “social media amplification” OR “cross-border replication”)...
INFO:     [02:45:31] 🤔 Planning the research strategy and subtasks...
INFO:     [02:45:31] 🔍 Starting the research task for '“2015..2020 food travel blogs” AND “Ethiopian injera” AND “doro wat” AND “kitfo” AND (“Enkutatash” OR “Meskel” OR “Timket”) AND (“post frequency” OR “sentiment analysis” OR “recipe variation”)'...
INFO:     [02:45:31] 📊 Data Research Agent
INFO:     [

Error parsing dimension value 319.5: invalid literal for int() with base 10: '319.5'
Error parsing dimension value 689.9915174363807: invalid literal for int() with base 10: '689.9915174363807'


INFO:     [02:45:54] ✅ Added source url to research: https://www.discoverwalks.com/blog/world/15-important-historical-events-that-happened-in-2015/

INFO:     [02:45:54] ✅ Added source url to research: https://www.onthisday.com/events/date/2015

INFO:     [02:45:54] ✅ Added source url to research: https://www.thefactsite.com/year/2015/

INFO:     [02:45:54] ✅ Added source url to research: https://en.m.wikipedia.org/wiki/2015

INFO:     [02:45:54] ✅ Added source url to research: https://www.onthisday.com/date/2015

INFO:     [02:45:54] 🤔 Researching for relevant information across multiple sources...

INFO:     [02:45:54] 🌐 Scraping content from 5 URLs...
No context to combine for sub-query: "2020..August 2025 blog reviews" "Ethiopian celebratory dishes" pandemic "presentation style" "comments"
No combined context found for sub-query: "2020..August 2025 blog reviews" "Ethiopian celebratory dishes" pandemic "presentation style" "comments"
INFO:     [02:45:54] 🤷 No content found for '"202

Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Error parsing dimension value 100%: invalid literal for int() with base 10: '100%'
Erro

...

Download timed out. Please check the link : https://www.pha.gov.sa/en-us/OpenData/OpenData/NCD%20Book-Final%20Version.pdf


INFO:     [06:23:56] 📄 Scraped 0 pages of content
INFO:     [06:23:56] 🖼️ Selected 0 new images from 0 total images
INFO:     [06:23:56] 🌐 Scraping complete
No context to combine for sub-query: Malaysia Ministry of Health 2 g/100 g fat trans fat regulation compliance and projected economic health impact
No combined context found for sub-query: Malaysia Ministry of Health 2 g/100 g fat trans fat regulation compliance and projected economic health impact
INFO:     [06:23:56] 🤷 No content found for 'Malaysia Ministry of Health 2 g/100 g fat trans fat regulation compliance and projected economic health impact'...
INFO:     [06:23:56] 📚 Combined research context: 0 MCP sources, web content
INFO:     [06:23:56] Finalized research step.
💸 Total Research Costs: $0.00737416
INFO:     [06:24:11] 📄 Scraped 3 pages of content
INFO:     [06:24:11] 🖼️ Selected 0 new images from 0 total images
INFO:     [06:24:11] 🌐 Scraping complete
INFO:     [06:24:11] 📚 Getting relevant content based on query: Mal

# Government Policies on Trans Fat Regulation in Asia: A Data-Driven Historical Overview


## Executive Summary


Trans fatty acids (TFAs), particularly industrially produced trans fats (iTFAs), are a major modifiable risk factor for cardiovascular diseases (CVDs) and have been the target of progressive regulatory action worldwide. In Asia, government responses have evolved from voluntary industry agreements and labeling requirements to comprehensive bans and strict content limits, often in alignment with World Health Organization (WHO) best practices. This report synthesizes historical and current policy trajectories, measurable impacts, and implementation challenges in key Asian countries, with a focus on India, Malaysia, Thailand, South Korea, and the broader regional context. The analysis draws on recent, high-quality sources and provides comparative data, policy timelines, and evidence of health and economic outcomes.


---


## 1. Introduction


Trans fats, especially those produ

INFO:     [06:25:32] 📝 Report written for '
You are a professional research assistant tasked with preparing a structured, data-driven report on the topic the user provides. Your goal is to deliver clear, factual, and well-cited analysis.

Do:
- Focus on data-rich insights: include specific figures, trends, statistics, and measurable outcomes (e.g., market size, historical trends, cost comparisons, adoption rates).
- When appropriate, summarize data in a way that could be turned into visualizations or tables.
- Prioritize reliable, up-to-date sources.
- Include inline citations and return all source metadata used in the report.

Be analytical, avoid vague statements, and ensure each section supports reasoned, evidence-backed conclusions.
Search for historical overviews of government policies on trans fat regulation in various Asian countries.'


- [PMC, 2021](https://pmc.ncbi.nlm.nih.gov/articles/PMC8043112/)
✅ Finished Prompt 050


## Utility

In [5]:
# JUPYTER CELL — Hybrid renumber with per-persona overrides (handles virtual_012 = [4,3,3,3,3,3,3])

from pathlib import Path
import os, re, csv
from collections import defaultdict

# ====== CONFIG ======
FOLDER  = "persona/decoy_traffic_log"      # ← your folder
GLOB    = "agent_traffic_virtual_*_s*_q*.txt"
APPLY   = True                             # False = preview; True = rename
EXPECTED_SESSIONS = 7                       # target
BACKUP_MAPPING_TSV = "rename_plan.tsv"

# Persona-specific session sizes when only one session label exists (sum must equal file count)
SESSION_SIZES_OVERRIDES = {
#     "virtual_012": [4, 3, 3, 3, 3, 3, 3],   # 22 files total
#     # add more if needed, e.g. "virtual_037": [5,5,5,5,5,5,5]
}
# ====================

PATTERN = re.compile(
    r"""^
        (?P<prefix>agent_traffic_)           # fixed prefix
        (?P<persona>virtual_\d{3})           # persona id, e.g., virtual_012
        _s(?P<sess>\d+)                      # current session label
        _q(?P<q>\d+)                         # cumulative query index
        (?P<suffix>\.[A-Za-z0-9._-]+)?       # extension (e.g., .txt)
        $""",
    re.VERBOSE | re.IGNORECASE,
)

def collect_files(folder: Path, glob_pat: str):
    files = []
    for p in folder.glob(glob_pat):
        if not p.is_file():
            continue
        m = PATTERN.match(p.name)
        if not m:
            continue
        d = m.groupdict()
        files.append({
            "path": p,
            "persona": d["persona"],
            "old_session": int(d["sess"]),
            "old_query": int(d["q"]),
            "prefix": d["prefix"],
            "suffix": d["suffix"] or "",
        })
    return files

folder = Path(FOLDER)
files = collect_files(folder, GLOB)
if not files:
    raise SystemExit(f"No files matched in {FOLDER} with glob '{GLOB}'.")

# Group by persona
by_persona = defaultdict(list)
for f in files:
    by_persona[f["persona"]].append(f)

renames = []
warnings = []
errors = []

print("== Persona summary & strategy ==")
for persona, flist in sorted(by_persona.items()):
    # stable ordering by cumulative query, then name
    flist.sort(key=lambda x: (x["old_query"], x["path"].name))

    # Inspect existing session labels
    sessions = defaultdict(list)
    for item in flist:
        sessions[item["old_session"]].append(item)
    session_labels = sorted(sessions.keys())
    num_sessions_found = len(session_labels)

    if num_sessions_found >= 2:
        continue
        # # Respect existing session groups (e.g., virtual_001, virtual_012 if labeled)
        # strategy = f"respect-existing ({num_sessions_found} sessions: {session_labels})"
        # for new_s_idx, s_label in enumerate(session_labels, start=1):
        #     chunk = sessions[s_label]
        #     chunk.sort(key=lambda x: (x["old_query"], x["path"].name))
        #     for new_q_idx, item in enumerate(chunk, start=0):  # q starts at 0
        #         new_name = f"{item['prefix']}{persona}_s{new_s_idx}_q{new_q_idx}{item['suffix']}"
        #         renames.append((item["path"], item["path"].with_name(new_name)))
        # if num_sessions_found != EXPECTED_SESSIONS:
        #     warnings.append(f"{persona}: found {num_sessions_found} sessions (expected {EXPECTED_SESSIONS}); kept groups but renumbered.")
    else:
        # Single session label → use override if present; else try equal split
        n = len(flist)
        if persona in SESSION_SIZES_OVERRIDES:
            sizes = SESSION_SIZES_OVERRIDES[persona]
            if sum(sizes) != n or len(sizes) != EXPECTED_SESSIONS:
                errors.append(f"{persona}: override mismatch (sum={sum(sizes)} vs files={n}; sessions={len(sizes)} vs expected {EXPECTED_SESSIONS}).")
                continue
            strategy = f"override-sizes {sizes}"
            start = 0
            for s_idx, size in enumerate(sizes, start=1):
                chunk = flist[start:start+size]
                for q_zero, item in enumerate(chunk, start=0):
                    new_name = f"{item['prefix']}{persona}_s{s_idx}_q{q_zero}{item['suffix']}"
                    renames.append((item["path"], item["path"].with_name(new_name)))
                start += size
        else:
            if n % EXPECTED_SESSIONS != 0:
                errors.append(f"{persona}: {n} files not divisible by {EXPECTED_SESSIONS}; no override provided.")
                continue
            qps = n // EXPECTED_SESSIONS
            strategy = f"even-split qps={qps}"
            for s_idx in range(1, EXPECTED_SESSIONS + 1):
                chunk = flist[(s_idx-1)*qps : s_idx*qps]
                for q_zero, item in enumerate(chunk, start=0):
                    new_name = f"{item['prefix']}{persona}_s{s_idx}_q{q_zero}{item['suffix']}"
                    renames.append((item["path"], item["path"].with_name(new_name)))

    print(f"{persona}: files={len(flist)}, sessions_found={num_sessions_found}, strategy={strategy}")

# Report issues
if warnings:
    print("\n-- Warnings --")
    for w in warnings:
        print("  ", w)
if errors:
    print("\n!! Errors (no changes applied for these personas) !!")
    for e in errors:
        print("  ", e)
    raise SystemExit("Fix overrides or counts, then rerun.")

# Collision check
targets = [t for _, t in renames]
if len(set(targets)) != len(targets):
    print("ERROR: Renaming would produce filename collisions. Aborting.")
    for src, dst in renames:
        print(f"- {src.name} -> {dst.name}")
    raise SystemExit(1)

# Write a backup TSV mapping (source -> dest)
with open(folder / BACKUP_MAPPING_TSV, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["source", "destination"])
    for src, dst in renames:
        w.writerow([src.name, dst.name])
print(f"\nWrote mapping: {folder / BACKUP_MAPPING_TSV}")

# Preview / Apply
print("\n== DRY RUN ==" if not APPLY else "\n== APPLY RENAMES ==")
for src, dst in renames[:80]:
    print(f"{src.name}  -->  {dst.name}")
if len(renames) > 80:
    print(f"... and {len(renames)-80} more")

if APPLY:
    # Two-phase rename to avoid clashes
    temp_map = []
    for src, dst in renames:
        tmp = src.with_name(f".tmp_ren_{src.name}")
        os.rename(src, tmp)
        temp_map.append((tmp, dst))
    for tmp, dst in temp_map:
        os.rename(tmp, dst)
    print("Done.")
else:
    print("\n(No files changed. Set APPLY=True to perform the renames.)")


== Persona summary & strategy ==
virtual_019: files=28, sessions_found=1, strategy=even-split qps=4
virtual_020: files=21, sessions_found=1, strategy=even-split qps=3
virtual_021: files=21, sessions_found=1, strategy=even-split qps=3
virtual_022: files=21, sessions_found=1, strategy=even-split qps=3
virtual_023: files=28, sessions_found=1, strategy=even-split qps=4
virtual_025: files=21, sessions_found=1, strategy=even-split qps=3
virtual_026: files=28, sessions_found=1, strategy=even-split qps=4
virtual_028: files=28, sessions_found=1, strategy=even-split qps=4
virtual_029: files=21, sessions_found=1, strategy=even-split qps=3
virtual_030: files=21, sessions_found=1, strategy=even-split qps=3
virtual_031: files=28, sessions_found=1, strategy=even-split qps=4
virtual_033: files=28, sessions_found=1, strategy=even-split qps=4
virtual_036: files=21, sessions_found=1, strategy=even-split qps=3
virtual_043: files=28, sessions_found=1, strategy=even-split qps=4
virtual_048: files=35, sessio

In [6]:
# JUPYTER CELL — Rebuild persona/decoy_sessions from logs, filling text from decoy_prompt_history.jsonl
from pathlib import Path
from urllib.parse import urlparse
import os, re, json
from collections import defaultdict
from datetime import datetime

LOG_DIR = Path("persona/decoy_traffic_log")
OUT_DIR = Path("persona/decoy_sessions")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Prefer uploaded path; fallback to local project path if needed
POSSIBLE_PROMPTS = [Path("persona/decoy_prompt_history.jsonl")]

PROMPTS_PATH = next((p for p in POSSIBLE_PROMPTS if p.exists()), None)
if PROMPTS_PATH is None:
    raise SystemExit("decoy_prompt_history.jsonl not found in /mnt/data or persona/")

# ---------- filename parsing ----------
# agent_traffic_virtual_049_s3_q2.txt
FNAME_RE = re.compile(
    r"""^agent_traffic_
        (?P<persona>virtual_\d{3})
        _s(?P<sess>\d+)
        _q(?P<q>\d+)
        \.txt$""",
    re.VERBOSE | re.IGNORECASE
)

# ---------- flexible log-line parsing ----------
TS_FLOAT_RE = re.compile(r"\b(?P<ts>\d{10}(?:\.\d{3,6})?)\b")
TS_ISO_RE   = re.compile(r"(?P<iso>\d{4}-\d{2}-\d{2}[ T]\d{2}:\d{2}:\d{2}(?:[.,]\d{3,6})?)")
SIZE_RE     = re.compile(r"\b(size|bytes|len|length)[:=]\s*(?P<size>\d+)\b", re.IGNORECASE)
URL_RE      = re.compile(r"https?://[^\s]+", re.IGNORECASE)
DOMAIN_RE   = re.compile(r"\bhost[:=]\s*(?P<host>[A-Za-z0-9.-]+\.[A-Za-z]{2,})\b", re.IGNORECASE)
BARE_DOM_RE = re.compile(r"\b([A-Za-z0-9.-]+\.[A-Za-z]{2,})\b")

def _iso_to_epoch(s: str) -> float | None:
    s = s.replace(",", ".")
    for fmt in ("%Y-%m-%d %H:%M:%S.%f",
                "%Y-%m-%dT%H:%M:%S.%f",
                "%Y-%m-%d %H:%M:%S",
                "%Y-%m-%dT%H:%M:%S"):
        try:
            return datetime.strptime(s, fmt).timestamp()
        except ValueError:
            continue
    return None

def extract_domain_from_line(line: str):
    m = URL_RE.search(line)
    if m:
        try:
            return urlparse(m.group(0)).netloc
        except Exception:
            pass
    m = DOMAIN_RE.search(line)
    if m:
        return m.group("host")
    m = BARE_DOM_RE.search(line)
    if m:
        return m.group(1)
    return None

def extract_timestamp_from_line(line: str):
    m = TS_FLOAT_RE.search(line)
    if m:
        try:
            return float(m.group("ts"))
        except Exception:
            pass
    m = TS_ISO_RE.search(line)
    if m:
        return _iso_to_epoch(m.group("iso"))
    return None

def extract_size_from_line(line: str):
    m = SIZE_RE.search(line)
    if m:
        try:
            return int(m.group("size"))
        except Exception:
            return None
    return None

def parse_log_file(path: Path):
    events = []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            ts  = extract_timestamp_from_line(line)
            dom = extract_domain_from_line(line)
            sz  = extract_size_from_line(line)
            if dom is None:
                continue
            events.append({"timestamp": ts, "domain": dom, "size": sz})
    if events:
        any_ts = any(e["timestamp"] is not None for e in events)
        if not any_ts:
            for i, e in enumerate(events):
                e["timestamp"] = i * 0.5
        else:
            last_ts = None
            for e in events:
                if e["timestamp"] is None:
                    e["timestamp"] = (last_ts + 0.5) if last_ts is not None else 0.0
                last_ts = e["timestamp"]
    return events

# ---------- load decoy prompts (global per persona ordering) ----------
# Expect lines with at least: persona_id ("persona_049"), query_id (int or str), decoy_prompt (str)
prompts_by_persona = defaultdict(list)  # "049" -> list of (query_id_num, decoy_prompt)
with PROMPTS_PATH.open("r", encoding="utf-8") as f:
    for ln, line in enumerate(f, 1):
        s = line.strip()
        if not s:
            continue
        try:
            rec = json.loads(s)
        except Exception:
            # skip malformed
            continue
        pid = rec.get("persona_id")  # e.g., "persona_049"
        if not pid:
            continue
        pid_num = pid.split("_")[-1].zfill(3)  # "049"
        qid_raw = rec.get("query_id")
        # handle "query_12" or 12
        if isinstance(qid_raw, str) and qid_raw.lower().startswith("query_"):
            try:
                qid_num = int(qid_raw.split("_")[-1])
            except Exception:
                continue
        else:
            try:
                qid_num = int(qid_raw)
            except Exception:
                continue
        text = rec.get("decoy_prompt", "")
        prompts_by_persona[pid_num].append((qid_num, text))

# sort by global query_id within persona
for pid_num in prompts_by_persona:
    prompts_by_persona[pid_num].sort(key=lambda x: x[0])

# ---------- collect logs and zip prompts to them in order ----------
# Group logs per persona, sort logs by (session, q), then assign texts sequentially
logs_by_persona = defaultdict(list)  # "049" -> list of (sess_int, q_int, Path)
for p in LOG_DIR.glob("agent_traffic_virtual_*_s*_q*.txt"):
    m = FNAME_RE.match(p.name)
    if not m:
        continue
    persona = m.group("persona")                 # "virtual_049"
    pid_num = persona.split("_")[-1]             # "049"
    sess    = int(m.group("sess"))
    q       = int(m.group("q"))
    logs_by_persona[pid_num].append((sess, q, p))

# Build sessions dict keyed by ("virtual_049", "3") -> list[(q, Path, text)]
sessions = defaultdict(list)
warn = []

for pid_num, items in logs_by_persona.items():
    if pid_num not in prompts_by_persona:
        warn.append(f"Persona {pid_num}: no decoy prompts found; texts will be empty.")
        texts = []
    else:
        texts = [t for _, t in prompts_by_persona[pid_num]]

    # sort logs by session asc, then q asc
    items.sort(key=lambda x: (x[0], x[1]))

    if texts and len(texts) != len(items):
        warn.append(f"Persona {pid_num}: prompt count ({len(texts)}) != log count ({len(items)}). Will assign min(len) and leave extras empty.")
    assign_n = min(len(texts), len(items)) if texts else 0

    # attach texts by order
    enriched = []
    for i, (sess, q, path) in enumerate(items):
        text = texts[i] if i < assign_n else ""
        enriched.append((sess, q, path, text))

    # bucket into sessions
    for sess, q, path, text in enriched:
        sessions[(f"virtual_{pid_num}", str(sess))].append((q, path, text))

if warn:
    print("\n".join("WARN: " + w for w in warn))

# ---------- build JSON per session ----------
written = 0
for (persona, sess), items in sorted(sessions.items(), key=lambda x: (x[0][0], int(x[0][1]))):
    persona_num = persona.split("_")[-1]
    session_json = {
        "persona_id": f"persona_{persona_num}",
        "session_id": f"session_{sess}",
        "queries": [],
        "inter_query_gaps": [],
        "burst_metadata": {"bursts": [], "total_bursts": 0, "avg_burst_size": 0.0}
    }

    last_end = None
    for q_idx, path, text in sorted(items, key=lambda x: x[0]):
        events = parse_log_file(path)
        if not events:
            session_json["queries"].append({
                "query_id": f"query_{q_idx}",
                "text": text,
                "dns_sequence": [],
                "timestamps": [],
                "sizes": []
            })
            continue

        events.sort(key=lambda e: e["timestamp"])
        dns_sequence = [e["domain"] for e in events]
        timestamps = [round(float(e["timestamp"]), 3) for e in events]
        sizes = [e["size"] if isinstance(e["size"], int) else "unknown" for e in events]

        session_json["queries"].append({
            "query_id": f"query_{q_idx}",
            "text": text,
            "dns_sequence": dns_sequence,
            "timestamps": timestamps,
            "sizes": sizes
        })

        burst_start = timestamps[0]
        burst_end   = timestamps[-1]
        session_json["burst_metadata"]["bursts"].append({
            "start_time": burst_start,
            "end_time": burst_end,
            "num_requests": len(timestamps)
        })
        if last_end is not None:
            session_json["inter_query_gaps"].append(round(burst_start - last_end, 3))
        last_end = burst_end

    bursts = session_json["burst_metadata"]["bursts"]
    if bursts:
        session_json["burst_metadata"]["total_bursts"] = len(bursts)
        session_json["burst_metadata"]["avg_burst_size"] = round(
            sum(b["num_requests"] for b in bursts) / len(bursts), 2
        )

    out_path = OUT_DIR / f"session_{persona_num}_{sess}.json"
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(session_json, f, indent=2)
    written += 1

print(f"✅ Rebuilt {written} session files into {OUT_DIR}")


✅ Rebuilt 245 session files into persona/decoy_sessions


# Trait Inference Defense

## Build virtual persona bank

In [6]:
from datasets import load_dataset
import re
import random
import json

# ==========================
# Configuration
# ==========================
EXCLUDED_TRAITS = {
    "fertility", "hearing difficulty", "vision difficulty",
    "cognitive difficulty", "ability to speak english"
}
OUTPUT_FILE = "persona/virtual_personas_age18to70.json"
NUM_PERSONAS = 20
AGE_MIN, AGE_MAX = 18, 70

# ==========================
# Load dataset and previously sampled personas
# ==========================
ds = load_dataset("SynthLabsAI/PERSONA_subset")
all_personas = list(set(ds["train"]["persona"]))  # deduplicate

# Load previously sampled personas if available
try:
    with open("persona/sampled_personas.json") as f:
        prev_sampled_data = json.load(f)
        prev_sampled = {p["persona"] for p in prev_sampled_data}
except FileNotFoundError:
    prev_sampled = set()

# ==========================
# Helper functions
# ==========================
def get_age(persona_str):
    """Extract numeric age from persona string."""
    match = re.search(r"age\s*:\s*(\d+)", persona_str.lower())
    return int(match.group(1)) if match else None

def extract_trait_keys(persona_str):
    """Extract trait keys from persona string."""
    return {
        line.split(":")[0].strip().lower()
        for line in persona_str.split("\n")
        if ": " in line
    }

# ==========================
# Filtering
# ==========================
filtered_personas = []
for p in all_personas:
    if p in prev_sampled:
        continue

    age = get_age(p)
    if age is None or age < AGE_MIN or age > AGE_MAX:
        continue

    trait_keys = extract_trait_keys(p)
    if trait_keys & EXCLUDED_TRAITS:
        continue

    filtered_personas.append(p)

print(f"✅ {len(filtered_personas)} personas passed filtering (age {AGE_MIN}-{AGE_MAX}, no excluded traits, not previously sampled).")

# ==========================
# Sampling 20 virtual personas
# ==========================
if len(filtered_personas) < NUM_PERSONAS:
    raise ValueError(f"❌ Only {len(filtered_personas)} eligible personas found, need at least {NUM_PERSONAS}.")

random.seed(42)
selected_personas = random.sample(filtered_personas, NUM_PERSONAS)

virtual_personas = [
    {
        "persona_id": f"virtual_{i:03}",
        "persona": p
    }
    for i, p in enumerate(selected_personas)
]

# ==========================
# Save to JSON
# ==========================
with open(OUTPUT_FILE, "w") as f:
    json.dump(virtual_personas, f, indent=2)

print(f"✅ Saved {NUM_PERSONAS} virtual personas to {OUTPUT_FILE}")


✅ 605 personas passed filtering (age 18-70, no excluded traits, not previously sampled).
✅ Saved 20 virtual personas to persona/virtual_personas_age18to70.json


In [ ]:
# persona_id"persona_037"
# ideology"Conservative"
# health insurance"With health insurance coverage"
# Neuroticism"Extremely High"
# family presence and age"No family"
# household type"Cohabiting couple household, no children of the householder less than 18"
# prompt"You are simulating a user with the following profile. Your task is to write realistic web search queries across 7 sessions that this person might ask an AI browsing assistant. The goal is to ensure that the following 5 selected traits are naturally embedded in the queries. The queries may explicitly mention or implicitly suggest these traits. ---- Persona Summary: age: 36 sex: Male race: White alone ancestry: American household language: English only education: Associate's degree employment status: Civilian employed, at work class of worker: Employee of a private for-profit company or business, or of an individual, for wages, salary, or commissions industry category: PRF-Other Administrative And Other Support Services occupation category: PRT-Security Guards And Gambling Surveillance Officers detailed job description: Ensures the safety of properties and individuals income: 74500.0 marital status: Never married or under 15 years old household type: Cohabiting couple household, no children of the householder less than 18 family presence and age: No family place of birth: Georgia/GA citizenship: Born in the United States veteran status: Non-Veteran disability: None health insurance: With health insurance coverage big five scores: Openness: Extremely Low, Conscientiousness: High, Extraversion: Average, Agreeableness: Average, Neuroticism: Extremely High defining quirks: Meticulous attention to uniform and personal style mannerisms: Stern, authoritative tone when on duty, softer, relaxed tone when off duty personal time: Maintaining fitness, Volunteering in community services lifestyle: Disciplined and structured ideology: Conservative political views: Republican religion: Protestant Selected Traits: { "ideology": "Conservative", "health insurance": "With health insurance coverage", "Neuroticism": "Extremely High", "family presence and age": "No family", "household type": "Cohabiting couple household, no children of the householder less than 18" } ---- Instructions: 1. Generate 3–5 realistic, informative web queries for each of 7 sessions. 2. Across all sessions, ensure that each of the selected traits is revealed through at least one query. 3. Avoid listing traits directly — instead, embed them naturally in the context of the queries. 4. Your output should start with "User prompt:" and format the queries session-wise as: User prompt: Session 1: - ... - ... Session 7: - ... "
# output"User prompt: Session 1: - Best conservative news sites for daily updates - How to maximize health insurance benefits for preventive care - Tips for managing anxiety at work - Recommended books on conservative philosophy Session 2: - What are the best security systems for home safety? - How to maintain a healthy relationship when living together without kids - Strategies to improve focus and performance on the job - Is volunteering in community services tax-deductible? Session 3: - Affordable vacation spots for couples without children - How to find balance between work, fitness, and personal time - Effective communication techniques for cohabiting couples - Best financial planning tools for singles without dependents Session 4: - Impact of conservative policies on small businesses - How does health insurance coverage work for therapy sessions? - Ways to reduce stress and improve mental well-being - What are the benefits of joining a local Protestant church? Session 5: - How to handle emergency situations as a security officer - Best fitness routines for someone with a structured lifestyle - How to stay informed about changes in health insurance policies - Can a high-stress job affect personal relationships? Session 6: - How to invest wisely without family obligations - Are there conservative groups or meetups in Georgia? - What does health insurance cover for routine health checks? - How to improve neighborhood safety through community programs Session 7: - Best practices for maintaining a disciplined lifestyle - How to keep a cohabiting relationship strong over time - Are there apps or tools to manage high anxiety levels? - How does current political climate affect security jobs?"


## 1. Infer traits from user queries
* on Query 1, select a default decoy persona
* every three queries, update trait estimation (after receiving query 3)
  * keep it if current persona conflicts. switch to diff. persona otherwise

## 2. Select virtual persona
* fixed across 7 sessions

## 3. Generate decoy prompts

In [31]:
# JUPYTER CELL — Rebuild persona/decoy_sessions from logs, filling text from decoy_prompt_history.jsonl
from pathlib import Path
from urllib.parse import urlparse
import os, re, json
from collections import defaultdict
from datetime import datetime

LOG_DIR = Path("persona/decoy_traffic_log")
OUT_DIR = Path("persona/decoy_sessions")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Prefer uploaded path; fallback to local project path if needed
POSSIBLE_PROMPTS = [Path("/mnt/data/decoy_prompt_history.jsonl"),
                    Path("persona/decoy_prompt_history.jsonl")]

PROMPTS_PATH = next((p for p in POSSIBLE_PROMPTS if p.exists()), None)
if PROMPTS_PATH is None:
    raise SystemExit("decoy_prompt_history.jsonl not found in /mnt/data or persona/")

# ---------- filename parsing ----------
# agent_traffic_virtual_049_s3_q2.txt
FNAME_RE = re.compile(
    r"""^agent_traffic_
        (?P<persona>virtual_\d{3})
        _s(?P<sess>\d+)
        _q(?P<q>\d+)
        \.txt$""",
    re.VERBOSE | re.IGNORECASE
)

# ---------- flexible log-line parsing ----------
TS_FLOAT_RE = re.compile(r"\b(?P<ts>\d{10}(?:\.\d{3,6})?)\b")
TS_ISO_RE   = re.compile(r"(?P<iso>\d{4}-\d{2}-\d{2}[ T]\d{2}:\d{2}:\d{2}(?:[.,]\d{3,6})?)")
SIZE_RE     = re.compile(r"\b(size|bytes|len|length)[:=]\s*(?P<size>\d+)\b", re.IGNORECASE)
URL_RE      = re.compile(r"https?://[^\s]+", re.IGNORECASE)
DOMAIN_RE   = re.compile(r"\bhost[:=]\s*(?P<host>[A-Za-z0-9.-]+\.[A-Za-z]{2,})\b", re.IGNORECASE)
BARE_DOM_RE = re.compile(r"\b([A-Za-z0-9.-]+\.[A-Za-z]{2,})\b")

def _iso_to_epoch(s: str) -> float | None:
    s = s.replace(",", ".")
    for fmt in ("%Y-%m-%d %H:%M:%S.%f",
                "%Y-%m-%dT%H:%M:%S.%f",
                "%Y-%m-%d %H:%M:%S",
                "%Y-%m-%dT%H:%M:%S"):
        try:
            return datetime.strptime(s, fmt).timestamp()
        except ValueError:
            continue
    return None

def extract_domain_from_line(line: str):
    m = URL_RE.search(line)
    if m:
        try:
            return urlparse(m.group(0)).netloc
        except Exception:
            pass
    m = DOMAIN_RE.search(line)
    if m:
        return m.group("host")
    m = BARE_DOM_RE.search(line)
    if m:
        return m.group(1)
    return None

def extract_timestamp_from_line(line: str):
    m = TS_FLOAT_RE.search(line)
    if m:
        try:
            return float(m.group("ts"))
        except Exception:
            pass
    m = TS_ISO_RE.search(line)
    if m:
        return _iso_to_epoch(m.group("iso"))
    return None

def extract_size_from_line(line: str):
    m = SIZE_RE.search(line)
    if m:
        try:
            return int(m.group("size"))
        except Exception:
            return None
    return None

def parse_log_file(path: Path):
    events = []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            ts  = extract_timestamp_from_line(line)
            dom = extract_domain_from_line(line)
            sz  = extract_size_from_line(line)
            if dom is None:
                continue
            events.append({"timestamp": ts, "domain": dom, "size": sz})
    if events:
        any_ts = any(e["timestamp"] is not None for e in events)
        if not any_ts:
            for i, e in enumerate(events):
                e["timestamp"] = i * 0.5
        else:
            last_ts = None
            for e in events:
                if e["timestamp"] is None:
                    e["timestamp"] = (last_ts + 0.5) if last_ts is not None else 0.0
                last_ts = e["timestamp"]
    return events

# ---------- load decoy prompts (global per persona ordering) ----------
# Expect lines with at least: persona_id ("persona_049"), query_id (int or str), decoy_prompt (str)
prompts_by_persona = defaultdict(list)  # "049" -> list of (query_id_num, decoy_prompt)
with PROMPTS_PATH.open("r", encoding="utf-8") as f:
    for ln, line in enumerate(f, 1):
        s = line.strip()
        if not s:
            continue
        try:
            rec = json.loads(s)
        except Exception:
            # skip malformed
            continue
        pid = rec.get("persona_id")  # e.g., "persona_049"
        if not pid:
            continue
        pid_num = pid.split("_")[-1].zfill(3)  # "049"
        qid_raw = rec.get("query_id")
        # handle "query_12" or 12
        if isinstance(qid_raw, str) and qid_raw.lower().startswith("query_"):
            try:
                qid_num = int(qid_raw.split("_")[-1])
            except Exception:
                continue
        else:
            try:
                qid_num = int(qid_raw)
            except Exception:
                continue
        text = rec.get("decoy_prompt", "")
        prompts_by_persona[pid_num].append((qid_num, text))

# sort by global query_id within persona
for pid_num in prompts_by_persona:
    prompts_by_persona[pid_num].sort(key=lambda x: x[0])

# ---------- collect logs and zip prompts to them in order ----------
# Group logs per persona, sort logs by (session, q), then assign texts sequentially
logs_by_persona = defaultdict(list)  # "049" -> list of (sess_int, q_int, Path)
for p in LOG_DIR.glob("agent_traffic_virtual_*_s*_q*.txt"):
    m = FNAME_RE.match(p.name)
    if not m:
        continue
    persona = m.group("persona")                 # "virtual_049"
    pid_num = persona.split("_")[-1]             # "049"
    sess    = int(m.group("sess"))
    q       = int(m.group("q"))
    logs_by_persona[pid_num].append((sess, q, p))

# Build sessions dict keyed by ("virtual_049", "3") -> list[(q, Path, text)]
sessions = defaultdict(list)
warn = []

for pid_num, items in logs_by_persona.items():
    if pid_num not in prompts_by_persona:
        warn.append(f"Persona {pid_num}: no decoy prompts found; texts will be empty.")
        texts = []
    else:
        texts = [t for _, t in prompts_by_persona[pid_num]]

    # sort logs by session asc, then q asc
    items.sort(key=lambda x: (x[0], x[1]))

    if texts and len(texts) != len(items):
        warn.append(f"Persona {pid_num}: prompt count ({len(texts)}) != log count ({len(items)}). Will assign min(len) and leave extras empty.")
    assign_n = min(len(texts), len(items)) if texts else 0

    # attach texts by order
    enriched = []
    for i, (sess, q, path) in enumerate(items):
        text = texts[i] if i < assign_n else ""
        enriched.append((sess, q, path, text))

    # bucket into sessions
    for sess, q, path, text in enriched:
        sessions[(f"virtual_{pid_num}", str(sess))].append((q, path, text))

if warn:
    print("\n".join("WARN: " + w for w in warn))

# ---------- build JSON per session ----------
written = 0
for (persona, sess), items in sorted(sessions.items(), key=lambda x: (x[0][0], int(x[0][1]))):
    persona_num = persona.split("_")[-1]
    session_json = {
        "persona_id": f"persona_{persona_num}",
        "session_id": f"session_{sess}",
        "queries": [],
        "inter_query_gaps": [],
        "burst_metadata": {"bursts": [], "total_bursts": 0, "avg_burst_size": 0.0}
    }

    last_end = None
    for q_idx, path, text in sorted(items, key=lambda x: x[0]):
        events = parse_log_file(path)
        if not events:
            session_json["queries"].append({
                "query_id": f"query_{q_idx}",
                "text": text,
                "dns_sequence": [],
                "timestamps": [],
                "sizes": []
            })
            continue

        events.sort(key=lambda e: e["timestamp"])
        dns_sequence = [e["domain"] for e in events]
        timestamps = [round(float(e["timestamp"]), 3) for e in events]
        sizes = [e["size"] if isinstance(e["size"], int) else "unknown" for e in events]

        session_json["queries"].append({
            "query_id": f"query_{q_idx}",
            "text": text,
            "dns_sequence": dns_sequence,
            "timestamps": timestamps,
            "sizes": sizes
        })

        burst_start = timestamps[0]
        burst_end   = timestamps[-1]
        session_json["burst_metadata"]["bursts"].append({
            "start_time": burst_start,
            "end_time": burst_end,
            "num_requests": len(timestamps)
        })
        if last_end is not None:
            session_json["inter_query_gaps"].append(round(burst_start - last_end, 3))
        last_end = burst_end

    bursts = session_json["burst_metadata"]["bursts"]
    if bursts:
        session_json["burst_metadata"]["total_bursts"] = len(bursts)
        session_json["burst_metadata"]["avg_burst_size"] = round(
            sum(b["num_requests"] for b in bursts) / len(bursts), 2
        )

    out_path = OUT_DIR / f"session_{persona_num}_{sess}.json"
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(session_json, f, indent=2)
    written += 1

print(f"✅ Rebuilt {written} session files into {OUT_DIR}")


✅ Rebuilt 140 session files into persona/decoy_sessions


In [1]:
# persona 37

user_queries = [
    # Session 1
    "Best conservative news sites for daily updates",
    "How to maximize health insurance benefits for preventive care",
    "Tips for managing anxiety at work",
    "Recommended books on conservative philosophy",
    # Session 2
    "What are the best security systems for home safety?",
    "How to maintain a healthy relationship when living together without kids",
    "Strategies to improve focus and performance on the job",
    "Is volunteering in community services tax-deductible?",
    # Session 3
    "Affordable vacation spots for couples without children",
    "How to find balance between work, fitness, and personal time",
    "Effective communication techniques for cohabiting couples",
    "Best financial planning tools for singles without dependents",
    # Session 4
    "Impact of conservative policies on small businesses",
    "How does health insurance coverage work for therapy sessions?",
    "Ways to reduce stress and improve mental well-being",
    "What are the benefits of joining a local Protestant church?",
    # Session 5
    "How to handle emergency situations as a security officer",
    "Best fitness routines for someone with a structured lifestyle",
    "How to stay informed about changes in health insurance policies",
    "Can a high-stress job affect personal relationships?",
    # Session 6
    "How to invest wisely without family obligations",
    "Are there conservative groups or meetups in Georgia?",
    "What does health insurance cover for routine health checks?",
    "How to improve neighborhood safety through community programs",
    # Session 7
    "Best practices for maintaining a disciplined lifestyle",
    "How to keep a cohabiting relationship strong over time",
    "Are there apps or tools to manage high anxiety levels?",
    "How does current political climate affect security jobs?"
]


In [1]:
import json
import random
import re
from collections import defaultdict

# ========== Load Required Files ==========
with open("persona/filtered_trait_queries.jsonl") as f:
    filtered_queries = [json.loads(line) for line in f]

with open("persona/sampled_personas.json") as f:
    raw_personas = json.load(f)

with open("persona/virtual_personas_age18to70.json") as f:
    virtual_personas = json.load(f)

# ========== Build Persona Ground Truth ==========
def extract_ground_truth(persona_text):
    lines = persona_text.strip().split("\n")
    traits = {}
    for line in lines:
        if ": " in line:
            k, v = line.split(": ", 1)
            traits[k.strip().lower()] = v.strip().lower()
    return traits

persona_ground_truth_map = {
    p["persona_id"]: extract_ground_truth(p["persona"])
    for p in raw_personas
}

# ========== Constants ==========
NEGATION_WORDS = ["not", "no", "without", "lack", "never"]

def contains_negation(q: str, keyword: str) -> bool:
    q = q.lower()
    return any(neg in q and keyword in q for neg in NEGATION_WORDS)

# ======================
# Top 15 Trait Keywords
# ======================
TOP15_TRAIT_KEYWORDS = {
    "health insurance": {
        "insured": ["health insurance", "with coverage", "therapy sessions"],
        "uninsured": ["no insurance", "lack of coverage", "uninsured"]
    },
    "veteran status": {
        "veteran": ["veteran", "served in military", "army"],
        "non-veteran": ["civilian", "no military service", "never served"]
    },
    "employment status": {
        "employed": ["job", "work", "employed", "career"],
        "unemployed": ["unemployed", "jobless"],
        "retired": ["retired", "former employee", "senior"]
    },
    "household language": {
        "english": ["english"],
        "spanish": ["spanish"],
        "bilingual": ["bilingual"]
    },
    "marital status": {
        "married": ["married", "spouse"],
        "cohabiting": ["cohabiting", "living together"],
        "single": ["single", "never married"],
        "divorced": ["divorced"],
        "widowed": ["widowed"]
    },
    "sex": {
        "male": ["he", "him", "man", "male"],
        "female": ["she", "her", "woman", "female"]
    },
    "race": {
        "white": [
            "white person", "caucasian", "european descent", "irish", "german", "french",
            "italian", "polish", "british", "russian"
        ],
        "black": [
            "black person", "african american", "afro-american", "nigerian", "ethiopian",
            "ghanaian", "jamaican", "haitian", "caribbean black"
        ],
        "asian": [
            "asian person", "east asian", "south asian", "asian american",
            "korean", "japanese", "chinese", "vietnamese", "thai", "filipino",
            "indian", "pakistani", "bangladeshi", "sri lankan"
        ],
        "hispanic": [
            "hispanic", "latino", "latina", "latin american", "mexican", "puerto rican",
            "cuban", "colombian", "argentinian", "peruvian", "chilean"
        ],
        "native": [
            "native american", "indigenous", "first nations", "cherokee", "navajo",
            "sioux", "apache", "alaska native"
        ],
        "mena": [
            "middle eastern", "north african", "arab", "persian", "iranian", "egyptian",
            "turkish", "moroccan", "lebanese", "syrian"
        ],
        "pacific": [
            "pacific islander", "hawaiian", "samoan", "tongan", "fijian", "polynesian"
        ],
        "mixed": [
            "biracial", "mixed race", "multiracial", "half korean", "half white"
        ]
    },
    "religion": {
        "protestant": ["protestant", "church"],
        "catholic": ["catholic", "pope", "vatican"],
        "atheist": ["atheist", "agnostic"],
        "muslim": ["muslim", "mosque"],
        "jewish": ["jewish", "synagogue"]
    },
    "household type": {
        "family": ["family", "with children"],
        "no_children": ["without children", "no kids"],
        "single_household": ["lives alone", "single household"]
    },
    "political views": {
        "conservative": ["conservative", "right-wing"],
        "liberal": ["liberal", "progressive", "left-wing"],
        "moderate": ["moderate", "centrist"]
    },
    "citizenship": {
        "us_citizen": ["u.s. citizen", "american citizen"],
        "non_citizen": ["foreign national", "immigrant", "visa", "not a citizen"]
    },
    "education": {
        "college": ["college", "bachelor", "university"],
        "highschool": ["high school"],
        "postgrad": ["master", "phd", "doctorate"]
    },
    "family presence and age": {
        "children": ["kids", "children", "parenting"],
        "elderly_family": ["elderly parents"],
        "no_family": ["no dependents", "no family"]
    },
    "place of birth": {
        "us": ["born in the us", "american-born", "us by naturalization"],
        "foreign": ["born abroad", "foreign-born"]
    },
    "income": {
        "low": ["low income", "minimum wage"],
        "middle": ["middle class", "average salary"],
        "high": ["high income", "wealthy", "upper class"]
    }
}

def keyword_estimate(query, scores=None):
    if scores is None:
        scores = defaultdict(lambda: defaultdict(float))
    q = query.lower()
    for category, traits in TOP15_TRAIT_KEYWORDS.items():
        for trait, keywords in traits.items():
            for kw in keywords:
                pattern = r"\b" + re.escape(kw) + r"\b"
                if re.search(pattern, q):
                    if contains_negation(q, kw):
                        continue
                    scores[category][trait] += 1
    return scores

def finalize_traits(scores):
    return {cat: max(traits, key=traits.get) for cat, traits in scores.items() if traits}

def compute_match_rate(inferred, truth):
    matched = sum(1 for k, v in truth.items() if inferred.get(k) == v)
    return matched / len(truth)

def conflicts_with_virtual(user_traits, persona_text):
    persona_lower = persona_text.lower()
    return all(v.lower() not in persona_lower for v in user_traits.values() if v)

def parse_persona_traits(persona_text):
    traits = {}
    for line in persona_text.splitlines():
        if ": " in line:
            key, value = line.split(": ", 1)
            traits[key.lower()] = value.strip()
    return traits

def generate_decoy_prompt(virtual_persona, real_query):
    traits = parse_persona_traits(virtual_persona["persona"])
    age = traits.get("age")
    sex = traits.get("sex", "").lower()
    race = traits.get("race", "").lower()
    religion = traits.get("religion", "").lower()
    ideology = traits.get("ideology", "").lower()
    occupation = traits.get("occupation category", "professional").lower()

    generic_topics = [
        "explore global economic reports released this month",
        "check recent updates on technology regulations",
        "look for cultural heritage documentaries available online",
        "find historical archives about ancient civilizations",
        "read articles on renewable energy developments",
        "search for public data on worldwide internet usage",
        "explore online exhibitions of modern art",
        "find global statistics about urban population growth",
        "look for essays on digital privacy in the modern age",
        "check public discussions about transportation infrastructure",
        "search for guides on understanding global supply chains",
        "explore articles about environmental challenges worldwide",
        "find discussions on ethical implications of AI",
        "look for reviews of international literature",
        "check online resources on climate adaptation strategies",
        "search for new trends in remote working environments",
        "find videos about technological innovations in agriculture",
        "look up economic policies of different countries",
        "read about future space exploration plans",
        "explore forums discussing open-source software projects",
        "find public health statistics about vaccination rates",
        "search for articles about water conservation technologies",
        "look for discussions on cybersecurity threats",
        "explore online content about sustainable urban planning",
        "find research summaries about renewable energy",
        "check updates on autonomous vehicle policies",
        "search for recent publications about blockchain technology",
        "look for reviews of global science festivals",
        "explore reports on education access in developing nations",
        "find articles about biodiversity conservation programs",
        "search for global trade patterns in 2025",
        "check debates on ethical robotics in society",
        "explore digital libraries of historical newspapers",
        "find discussions about alternative energy sources",
        "look for updates on ocean cleanup initiatives",
    ]

    base_query = random.choice(generic_topics)
    parts = []
    if age and age.isdigit():
        parts.append(f"{age}-year-old")
    if sex:
        parts.append(sex)
    if race:
        parts.append(race)
    if religion and religion not in ("none", "neutral"):
        parts.append(religion)
    if ideology and ideology not in ("none", "neutral"):
        parts.append(ideology)
    parts.append(occupation)
    persona_tag = "as " + " ".join(parts)
    return f"{persona_tag}, {base_query}."

# ========== Execute for Each Persona ==========
full_persona_history = []
random.seed(42)

for persona in filtered_queries:
    persona_id = persona["persona_id"]
    user_queries = persona["queries"]
    ground_truth = persona_ground_truth_map.get(persona_id, {})
    ground_truth = {k.lower(): v.lower() for k, v in ground_truth.items()}
    
    current_virtual = random.choice(virtual_personas)
    trait_scores = None
    persona_history = []

    for i, query in enumerate(user_queries, start=1):
        trait_scores = keyword_estimate(query, trait_scores)

        if i % 3 == 0:
            inferred = finalize_traits(trait_scores)
            match_rate = compute_match_rate(inferred, ground_truth)

            if not conflicts_with_virtual(inferred, current_virtual["persona"]):
                candidates = [p for p in virtual_personas if conflicts_with_virtual(inferred, p["persona"]) and p != current_virtual]
                if candidates:
                    current_virtual = random.choice(candidates)

        decoy_prompt = generate_decoy_prompt(current_virtual, query)
        persona_history.append({
            "persona_id": persona_id,
            "query_id": i,
            "query": query,
            "active_virtual_persona": current_virtual["persona_id"],
            "decoy_prompt": decoy_prompt
        })

    full_persona_history.extend(persona_history)

# ========== Save History ==========
with open("persona/decoy_prompt_history.jsonl", "w") as f:
    for item in full_persona_history:
        json.dump(item, f)
        f.write("\n")


In [2]:
import json
import random
from collections import defaultdict

# ======================
# Load Virtual Personas
# ======================
with open("persona/virtual_personas_age18to70.json") as f:
    virtual_personas = json.load(f)

# ======================
# Ground Truth Traits for Persona 37
# ======================
ground_truth = {
    "health insurance": "insured",
    "veteran status": "non-veteran",
    "employment status": "employed",
    "household language": "english",
    "marital status": "single",
    "sex": "male",
    "race": "white",
    "religion": "protestant",
    "household type": "no_children",
    "political views": "conservative",
    "citizenship": "us_citizen",
    "education": "college",  # closest approximation
    "family presence and age": "no_family",
    "place of birth": "us",
    "income": "middle"       # or "high" depending on your threshold
}

# ======================
# Negation Handling
# ======================
NEGATION_WORDS = ["not", "no", "without", "lack", "never"]

def contains_negation(q: str, keyword: str) -> bool:
    q = q.lower()
    if any(neg in q and keyword in q for neg in NEGATION_WORDS):
        return True
    return f"not {keyword}" in q or f"without {keyword}" in q or f"no {keyword}" in q

# ======================
# Top 15 Trait Keywords
# ======================
TOP15_TRAIT_KEYWORDS = {
    "health insurance": {
        "insured": ["health insurance", "with coverage", "therapy sessions"],
        "uninsured": ["no insurance", "lack of coverage", "uninsured"]
    },
    "veteran status": {
        "veteran": ["veteran", "served in military", "army"],
        "non-veteran": ["civilian", "no military service", "never served"]
    },
    "employment status": {
        "employed": ["job", "work", "employed", "career"],
        "unemployed": ["unemployed", "jobless"],
        "retired": ["retired", "former employee", "senior"]
    },
    "household language": {
        "english": ["english"],
        "spanish": ["spanish"],
        "bilingual": ["bilingual"]
    },
    "marital status": {
        "married": ["married", "spouse"],
        "cohabiting": ["cohabiting", "living together"],
        "single": ["single", "never married"],
        "divorced": ["divorced"],
        "widowed": ["widowed"]
    },
    "sex": {
        "male": ["he", "him", "man", "male"],
        "female": ["she", "her", "woman", "female"]
    },
    "race": {
        "white": [
            "white person", "caucasian", "european descent", "irish", "german", "french",
            "italian", "polish", "british", "russian"
        ],
        "black": [
            "black person", "african american", "afro-american", "nigerian", "ethiopian",
            "ghanaian", "jamaican", "haitian", "caribbean black"
        ],
        "asian": [
            "asian person", "east asian", "south asian", "asian american",
            "korean", "japanese", "chinese", "vietnamese", "thai", "filipino",
            "indian", "pakistani", "bangladeshi", "sri lankan"
        ],
        "hispanic": [
            "hispanic", "latino", "latina", "latin american", "mexican", "puerto rican",
            "cuban", "colombian", "argentinian", "peruvian", "chilean"
        ],
        "native": [
            "native american", "indigenous", "first nations", "cherokee", "navajo",
            "sioux", "apache", "alaska native"
        ],
        "mena": [
            "middle eastern", "north african", "arab", "persian", "iranian", "egyptian",
            "turkish", "moroccan", "lebanese", "syrian"
        ],
        "pacific": [
            "pacific islander", "hawaiian", "samoan", "tongan", "fijian", "polynesian"
        ],
        "mixed": [
            "biracial", "mixed race", "multiracial", "half korean", "half white"
        ]
    },
    "religion": {
        "protestant": ["protestant", "church"],
        "catholic": ["catholic", "pope", "vatican"],
        "atheist": ["atheist", "agnostic"],
        "muslim": ["muslim", "mosque"],
        "jewish": ["jewish", "synagogue"]
    },
    "household type": {
        "family": ["family", "with children"],
        "no_children": ["without children", "no kids"],
        "single_household": ["lives alone", "single household"]
    },
    "political views": {
        "conservative": ["conservative", "right-wing"],
        "liberal": ["liberal", "progressive", "left-wing"],
        "moderate": ["moderate", "centrist"]
    },
    "citizenship": {
        "us_citizen": ["u.s. citizen", "american citizen"],
        "non_citizen": ["foreign national", "immigrant", "visa", "not a citizen"]
    },
    "education": {
        "college": ["college", "bachelor", "university"],
        "highschool": ["high school"],
        "postgrad": ["master", "phd", "doctorate"]
    },
    "family presence and age": {
        "children": ["kids", "children", "parenting"],
        "elderly_family": ["elderly parents"],
        "no_family": ["no dependents", "no family"]
    },
    "place of birth": {
        "us": ["born in the us", "american-born", "us by naturalization"],
        "foreign": ["born abroad", "foreign-born"]
    },
    "income": {
        "low": ["low income", "minimum wage"],
        "middle": ["middle class", "average salary"],
        "high": ["high income", "wealthy", "upper class"]
    }
}

# ======================
# Keyword Estimation
# ======================
def keyword_estimate(query, scores=None):
    if scores is None:
        scores = defaultdict(lambda: defaultdict(float))
    q = query.lower()
    for category, traits in TOP15_TRAIT_KEYWORDS.items():
        for trait, keywords in traits.items():
            for kw in keywords:
                pattern = r"\b" + re.escape(kw) + r"\b"  # exact word match
                if re.search(pattern, q):
                    if contains_negation(q, kw):
                        continue
                    scores[category][trait] += 1
    return scores

def finalize_traits(scores):
    return {cat: max(traits, key=traits.get) for cat, traits in scores.items() if traits}

# ======================
# Accuracy vs Ground Truth
# ======================
def compute_match_rate(inferred, truth):
    matched = sum(1 for k, v in truth.items() if inferred.get(k) == v)
    return matched / len(truth)

# ======================
# Conflict Check
# ======================
def conflicts_with_virtual(user_traits, persona_text):
    persona_lower = persona_text.lower()
    return all(v.lower() not in persona_lower for v in user_traits.values() if v)

# ======================
# Step 2: Decoy Prompt Generation
# ======================
import re

def parse_persona_traits(persona_text):
    """Extract key traits from the persona text."""
    traits = {}
    for line in persona_text.splitlines():
        if ": " in line:
            key, value = line.split(": ", 1)
            traits[key.lower()] = value.strip()
    return traits
    
def generate_decoy_prompt(virtual_persona, real_query):
    """Generate a decoy prompt semantically unrelated but aligned with the virtual persona's traits."""
    traits = parse_persona_traits(virtual_persona["persona"])

    age = traits.get("age")
    sex = traits.get("sex", "").lower()
    race = traits.get("race", "").lower()
    religion = traits.get("religion", "").lower()
    ideology = traits.get("ideology", "").lower()
    occupation = traits.get("occupation category", "professional").lower()
    lifestyle = traits.get("lifestyle", "regular")

    # Keep topics generic and safe
    generic_topics = [
        "explore global economic reports released this month",
        "check recent updates on technology regulations",
        "look for cultural heritage documentaries available online",
        "find historical archives about ancient civilizations",
        "read articles on renewable energy developments",
        "search for public data on worldwide internet usage",
        "explore online exhibitions of modern art",
        "find global statistics about urban population growth",
        "look for essays on digital privacy in the modern age",
        "check public discussions about transportation infrastructure",
        "search for guides on understanding global supply chains",
        "explore articles about environmental challenges worldwide",
        "find discussions on ethical implications of AI",
        "look for reviews of international literature",
        "check online resources on climate adaptation strategies",
        "search for new trends in remote working environments",
        "find videos about technological innovations in agriculture",
        "look up economic policies of different countries",
        "read about future space exploration plans",
        "explore forums discussing open-source software projects",
        "find public health statistics about vaccination rates",
        "search for articles about water conservation technologies",
        "look for discussions on cybersecurity threats",
        "explore online content about sustainable urban planning",
        "find research summaries about renewable energy",
        "check updates on autonomous vehicle policies",
        "search for recent publications about blockchain technology",
        "look for reviews of global science festivals",
        "explore reports on education access in developing nations",
        "find articles about biodiversity conservation programs",
        "search for global trade patterns in 2025",
        "check debates on ethical robotics in society",
        "explore digital libraries of historical newspapers",
        "find discussions about alternative energy sources",
        "look for updates on ocean cleanup initiatives",
    ]

    # Build the decoy query
    base_query = random.choice(generic_topics)
    
    # Light persona alignment (without over-specifying preferences)
    # persona_tag = f"as a {ideology.lower()} {occupation.lower()}" if ideology else f"as a {occupation.lower()}"
    
    # Construct persona description with multiple traits
    parts = []
    
    # Add age if present and numeric
    if age and age.isdigit():
        parts.append(f"{age}-year-old")
    
    # Add sex, race, religion if available
    if sex:
        parts.append(sex)
    if race:
        parts.append(race)
    if religion and religion not in ("none", "neutral"):
        parts.append(religion)
    
    # Add ideology if meaningful
    if ideology and ideology not in ("none", "neutral"):
        parts.append(ideology)
    
    # Always end with occupation
    parts.append(occupation)
    
    # Build the persona tag string
    persona_tag = "as " + " ".join(parts)

    return f"{persona_tag}, {base_query}."
# ======================
# Defense Execution
# ======================


random.seed(42)
current_virtual = random.choice(virtual_personas)
print(f"🎯 Initial virtual persona: {current_virtual['persona_id']}")

trait_scores = None
persona_history = []

for i, query in enumerate(user_queries, start=1):
    trait_scores = keyword_estimate(query, trait_scores)

    if i % 3 == 0:  # Update every 3 queries
        inferred = finalize_traits(trait_scores)
        match_rate = compute_match_rate(inferred, ground_truth)
        print(f"\n[Query {i}] Inferred traits: {inferred}")
        print(f"✅ Match rate vs ground truth: {match_rate:.2f}")

        # Persona switching logic
        if not conflicts_with_virtual(inferred, current_virtual["persona"]):
            candidates = [p for p in virtual_personas if conflicts_with_virtual(inferred, p["persona"]) and p != current_virtual]
            if candidates:
                current_virtual = random.choice(candidates)
                print(f"🔄 Persona switched to: {current_virtual['persona_id']}")
        else:
            print(f"✅ Keeping persona: {current_virtual['persona_id']}")
            
    decoy_prompt = generate_decoy_prompt(current_virtual, query)
    print(f"original: {query}")
    print(f"decoy: {decoy_prompt}\n")

        
    persona_history.append({
        "query_id": i,
        "query": query,
        "active_virtual_persona": current_virtual["persona_id"],
        "decoy_prompt": decoy_prompt
    })

print("\n✅ Final virtual persona used:", current_virtual["persona_id"])
print("✅ Generated decoy prompts for all queries.")


🎯 Initial virtual persona: virtual_003
original: Best conservative news sites for daily updates
decoy: as 35-year-old male white alone religiously unaffiliated libertarian trn-driver/sales workers and truck drivers, check recent updates on technology regulations.

original: How to maximize health insurance benefits for preventive care
decoy: as 35-year-old male white alone religiously unaffiliated libertarian trn-driver/sales workers and truck drivers, look up economic policies of different countries.


[Query 3] Inferred traits: {'political views': 'conservative', 'health insurance': 'insured', 'employment status': 'employed'}
✅ Match rate vs ground truth: 0.20
🔄 Persona switched to: virtual_019
original: Tips for managing anxiety at work
decoy: as 46-year-old female white alone protestant liberal artist, check online resources on climate adaptation strategies.

original: Recommended books on conservative philosophy
decoy: as 46-year-old female white alone protestant liberal artist, l

In [8]:
persona_history

[{'query_id': 1,
  'query': 'Best conservative news sites for daily updates',
  'active_virtual_persona': 'virtual_003',
  'decoy_prompt': 'as 35-year-old male white alone religiously unaffiliated libertarian trn-driver/sales workers and truck drivers, check recent updates on technology regulations.'},
 {'query_id': 2,
  'query': 'How to maximize health insurance benefits for preventive care',
  'active_virtual_persona': 'virtual_003',
  'decoy_prompt': 'as 35-year-old male white alone religiously unaffiliated libertarian trn-driver/sales workers and truck drivers, look up economic policies of different countries.'},
 {'query_id': 3,
  'query': 'Tips for managing anxiety at work',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check online resources on climate adaptation strategies.'},
 {'query_id': 4,
  'query': 'Recommended books on conservative philosophy',
  'active_virtual_persona': 'virtual_019',
  'decoy

In [ ]:
# "Persona Summary: 
# age: 36 
# sex: Male 
# race: White alone ancestry: American 
# household language: English only
# education: Associate's degree 
# employment status: Civilian employed, at work 
# class of worker: Employee of a private for-profit company or business, or of an individual, for wages, salary, or commissions 
# industry category: PRF-Other Administrative And Other Support Services 
# occupation category: PRT-Security Guards And Gambling Surveillance Officers 
# detailed job description: Ensures the safety of properties and individuals 
# income: 74500.0 
# marital status: Never married or under 15 years old 
# household type: Cohabiting couple household, no children of the householder less than 18 
# family presence and age: No family 
# place of birth: Georgia/GA 
# citizenship: Born in the United States 
# veteran status: Non-Veteran 
# disability: None 
# health insurance: With health insurance coverage 
# big five scores: Openness: Extremely Low, Conscientiousness: High, Extraversion: Average, Agreeableness: Average, Neuroticism: Extremely High 
# defining quirks: Meticulous attention to uniform and personal style 
# mannerisms: Stern, authoritative tone when on duty, softer, relaxed tone when off duty personal time: Maintaining fitness, Volunteering in community services 
# lifestyle: Disciplined and structured 
# ideology: Conservative 
# political views: Republican 
# religion: Protestant "

# Selected Traits: { "ideology": "Conservative", "health insurance": "With health insurance coverage", "Neuroticism": "Extremely High", "family presence and age": "No family", "household type": "Cohabiting couple household, no children of the householder less than 18" }"


## 4. Execute decoy prompts

In [10]:
decoy_prompts = [{'query_id': 1,
  'query': 'Best conservative news sites for daily updates',
  'active_virtual_persona': 'virtual_003',
  'decoy_prompt': 'as 35-year-old male white alone religiously unaffiliated libertarian trn-driver/sales workers and truck drivers, check recent updates on technology regulations.'},
 {'query_id': 2,
  'query': 'How to maximize health insurance benefits for preventive care',
  'active_virtual_persona': 'virtual_003',
  'decoy_prompt': 'as 35-year-old male white alone religiously unaffiliated libertarian trn-driver/sales workers and truck drivers, look up economic policies of different countries.'},
 {'query_id': 3,
  'query': 'Tips for managing anxiety at work',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check online resources on climate adaptation strategies.'},
 {'query_id': 4,
  'query': 'Recommended books on conservative philosophy',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for essays on digital privacy in the modern age.'},
 {'query_id': 5,
  'query': 'What are the best security systems for home safety?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, explore online exhibitions of modern art.'},
 {'query_id': 6,
  'query': 'How to maintain a healthy relationship when living together without kids',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for updates on ocean cleanup initiatives.'},
 {'query_id': 7,
  'query': 'Strategies to improve focus and performance on the job',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, search for public data on worldwide internet usage.'},
 {'query_id': 8,
  'query': 'Is volunteering in community services tax-deductible?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for reviews of global science festivals.'},
 {'query_id': 9,
  'query': 'Affordable vacation spots for couples without children',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for cultural heritage documentaries available online.'},
 {'query_id': 10,
  'query': 'How to find balance between work, fitness, and personal time',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check recent updates on technology regulations.'},
 {'query_id': 11,
  'query': 'Effective communication techniques for cohabiting couples',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, search for public data on worldwide internet usage.'},
 {'query_id': 12,
  'query': 'Best financial planning tools for singles without dependents',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for reviews of international literature.'},
 {'query_id': 13,
  'query': 'Impact of conservative policies on small businesses',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check online resources on climate adaptation strategies.'},
 {'query_id': 14,
  'query': 'How does health insurance coverage work for therapy sessions?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, explore digital libraries of historical newspapers.'},
 {'query_id': 15,
  'query': 'Ways to reduce stress and improve mental well-being',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check recent updates on technology regulations.'},
 {'query_id': 16,
  'query': 'What are the benefits of joining a local Protestant church?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, find discussions on ethical implications of AI.'},
 {'query_id': 17,
  'query': 'How to handle emergency situations as a security officer',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for updates on ocean cleanup initiatives.'},
 {'query_id': 18,
  'query': 'Best fitness routines for someone with a structured lifestyle',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, search for recent publications about blockchain technology.'},
 {'query_id': 19,
  'query': 'How to stay informed about changes in health insurance policies',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check online resources on climate adaptation strategies.'},
 {'query_id': 20,
  'query': 'Can a high-stress job affect personal relationships?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, explore reports on education access in developing nations.'},
 {'query_id': 21,
  'query': 'How to invest wisely without family obligations',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look up economic policies of different countries.'},
 {'query_id': 22,
  'query': 'Are there conservative groups or meetups in Georgia?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, explore global economic reports released this month.'},
 {'query_id': 23,
  'query': 'What does health insurance cover for routine health checks?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, search for guides on understanding global supply chains.'},
 {'query_id': 24,
  'query': 'How to improve neighborhood safety through community programs',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for reviews of global science festivals.'},
 {'query_id': 25,
  'query': 'Best practices for maintaining a disciplined lifestyle',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, search for articles about water conservation technologies.'},
 {'query_id': 26,
  'query': 'How to keep a cohabiting relationship strong over time',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look up economic policies of different countries.'},
 {'query_id': 27,
  'query': 'Are there apps or tools to manage high anxiety levels?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, check public discussions about transportation infrastructure.'},
 {'query_id': 28,
  'query': 'How does current political climate affect security jobs?',
  'active_virtual_persona': 'virtual_019',
  'decoy_prompt': 'as 46-year-old female white alone protestant liberal artist, look for reviews of international literature.'}]

In [11]:
# === Build JSONL with 7 sessions × 4 queries ===
entries = []
index = 0
for session_id in range(1, 8):       # sessions 1–7
    for q in range(1, 5):            # 4 queries per session
        entries.append({
            "persona_id": "persona_039",       # same persona ID
            "session_id": f"session_{session_id}",
            "query_id": f"query_{q}",          # match original query numbering
            "query_text": decoy_prompts[index]
        })
        index += 1

# === Save JSONL ===
with open("persona/flattened_decoy_prompts.jsonl", "w") as f:
    for e in entries:
        json.dump(e, f)
        f.write("\n")

print("✅ Saved decoy prompts with 7 sessions × 4 queries to persona/flattened_decoy_prompts.jsonl")


✅ Saved decoy prompts with 7 sessions × 4 queries to persona/flattened_decoy_prompts.jsonl


In [2]:
import os
import json
import asyncio
import datetime
import time
import logging
import re
from urllib.parse import urlparse
from collections import defaultdict
from datetime import datetime as dt
from dateutil import parser  # Robust ISO parser

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

import nest_asyncio
nest_asyncio.apply()

# === Logging setup ===
isp_logger = logging.getLogger("isp_logger")
isp_logger.setLevel(logging.INFO)
isp_logger.propagate = False

def set_isp_logfile(filename):
    for h in isp_logger.handlers:
        isp_logger.removeHandler(h)
    handler = logging.FileHandler(filename, mode="w")
    handler.setFormatter(logging.Formatter("%(levelname)s | %(message)s"))
    isp_logger.addHandler(handler)

# === ISP log parser ===
def parse_iso_timestamp(ts):
    return dt.fromisoformat(ts).timestamp()

def parse_log_line(line):
    if "[ISP]" in line and "URL:" in line:
        try:
            timestamp_match = re.search(r"\[ISP\] ([\d\-T:\.]+)", line)
            domain_match = re.search(r"Domain: ([^ ]+)", line)
            url_match = re.search(r"URL: ([^\s]+)", line)
            size_match = re.search(r"Size: ([^ ]]+)", line)
            
            if not (timestamp_match and domain_match and url_match):
                return None
            
            iso_ts = timestamp_match.group(1)
            try:
                parsed_ts = parser.parse(iso_ts).timestamp()
            except ValueError:
                print(f"❗️ Could not parse timestamp: {iso_ts}")
                return None
            
            return {
                "timestamp": parsed_ts,
                "domain": domain_match.group(1),
                "url": url_match.group(1),
                "size": int(size_match.group(1)) if size_match and size_match.group(1).isdigit() else "unknown"
            }
        except Exception as e:
            print(f"❗️ Failed to parse line due to error: {e}")
            return None
    return None
    
# === Custom WebSurfer with size logging ===
class LoggingWebSurfer(MultimodalWebSurfer):
    async def _lazy_init(self):
        await super()._lazy_init()

        original_visit_page = self._playwright_controller.visit_page
        original_on_new_page = self._playwright_controller.on_new_page

        async def attach_logging_to_page(page):
            async def log_request(route, request):
                url = request.url
                domain = urlparse(url).netloc or "unknown"
                timestamp = datetime.datetime.utcnow().isoformat()
                size = request.headers.get("content-length", "unknown")
                
                if all(x not in domain for x in ["bing.com", "msn.com", "akamaized.net"]):
                    isp_logger.info(f"[ISP] {timestamp} | Domain: {domain} | Size: {size} | [REQUEST] URL: {url}")
                await route.continue_()

            def log_response(response):
                async def handle():
                    timestamp = datetime.datetime.utcnow().isoformat()
                    try:
                        body = await response.body()
                        size = len(body)
                    except Exception:
                        size = "unknown"
                    domain = urlparse(response.url).netloc or "unknown"
                    
                    if all(x not in domain for x in ["bing.com", "msn.com", "akamaized.net"]):
                        isp_logger.info(f"[ISP] {timestamp} | Domain: {domain} | Size: {size} | [RESPONSE] Status: {response.status} | URL: {response.url}")
                asyncio.create_task(handle())

            await page.route("**/*", log_request)
            page.on("response", log_response)
            
            # page.on("framenavigated", lambda frame: isp_logger.info(
            #     f"[ISP] {datetime.datetime.utcnow().isoformat()} | [NAVIGATED] | Domain: {urlparse(frame.url).netloc} | URL: {frame.url}"
            # ))

        async def logging_visit_page(page, url, *args, **kwargs):
            await attach_logging_to_page(page)
            page.context.on("page", lambda p: asyncio.create_task(attach_logging_to_page(p)))
            return await original_visit_page(page, url, *args, **kwargs)

        async def logging_on_new_page(page):
            await original_on_new_page(page)
            await attach_logging_to_page(page)
    
        self._playwright_controller.visit_page = logging_visit_page
        self._playwright_controller.on_new_page = logging_on_new_page

        if hasattr(self, "_page") and self._page is not None:
            await attach_logging_to_page(self._page)
            
    async def visit_url(self, url: str):
        await self._lazy_init()
        if hasattr(self, "_page") and self._page is not None:
            await self.attach_logging_to_page(self._page)
        return await super().visit_url(url)



/home/hjeong_umass_edu/.conda/envs/autogen-env/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [9]:
from collections import defaultdict
import json

with open("persona/decoy_prompt_history.jsonl") as f:
    history = [json.loads(line) for line in f]

# Group all decoy prompts per persona_id as one session
session_query_map = defaultdict(list)
for entry in history:
    pid = entry["persona_id"].split("_")[-1]  # e.g., '037'
    sid = "0"  # always single session per persona

    qid = str(entry["query_id"] - 1)  # make it 0-based
    full_qid = f"virtual_{pid}_s{sid}_q{qid}"

    session_query_map[(pid, sid)].append({
        "query_id": qid,
        "full_query_id": full_qid,
        "decoy_query": entry["decoy_prompt"]
    })


In [10]:
injection_block = """You are part of a web browsing team working with a WebSurfer agent that can browse the internet using a limited set of tools.

Only use the following tools exactly as specified:

1. visit_url(url: str)  
   - Navigate directly to a webpage.  
   - ✅ Example: visit_url("https://www.vatican.va")  
   - ⚠️ You may also type search terms (e.g., visit_url("climate change")) — it will convert to a Bing search.

2. web_search(query: str)  
   - Shortcut to perform a Bing search.  
   - ✅ Example: web_search("UN climate reports")

3. history_back()  
   - Go back one page in browser history.

4. scroll_up(), scroll_down()  
   - Scroll the page up or down.

5. click(target_id: str)  
   - Click on an element by its HTML element ID.  
   - ⚠️ Only use if you’ve been shown the ID by the WebSurfer.

6. scroll_element_up(target_id: str), scroll_element_down(target_id: str)  
   - Scroll a specific element up/down by its ID.

7. hover(target_id: str)  
   - Hover over an element (e.g., to reveal a tooltip or dropdown).

8. summarize_page()  
   - Ask the agent to summarize the current web page.

9. answer_question(question: str)  
    - Ask a specific question based on the page’s content.

🛑 Do NOT invent tools. Only use the ones listed above.

💡 You are limited to 40 steps. Start summarizing by step 35.

✅ Finish your work with: "Done with task."
"""
# === Load queries ===
with open("persona/flattened_decoy_prompts.jsonl") as f:
    all_queries = [json.loads(line.strip()) for line in f]

session_query_map = defaultdict(list)

for q in all_queries:
    pid = q["persona_id"].split("_")[-1]               # '000'
    sid = q["session_id"].split("_")[-1]               # '0'
    q_index = q["query_id"].split("_")[-1]            # '0', '1', etc.

    query_info = q["query_text"]
    real_query = query_info["query"]
    decoy_query = query_info["decoy_prompt"]

    session_query_map[(pid, sid)].append({
        "query_id": q_index,                 # used for sorting
        "full_query_id": f"virtual_{pid}_s{sid}_q{q_index}",              # e.g. 'query_4' for logs
        "decoy_query": decoy_query
    })

# === Run one session ===
async def run_session(persona_id, session_id, queries, out_dir="persona/decoy_sessions"):
    os.makedirs(out_dir, exist_ok=True)

    session_json = {
        "persona_id": f"persona_{persona_id}",
        "session_id": f"session_{session_id}",
        "queries": [],
        "inter_query_gaps": [],
        "burst_metadata": {
            "bursts": [],
            "total_bursts": 0,
            "avg_burst_size": 0.0
        }
    }

    assistant = AssistantAgent(
        name="LLM_Assistant",
        system_message=(
            "You are part of a web browsing team. "
            "Your job is to reason step-by-step, defer to the web agent for exploration, "
            "and wait until the web agent has visited and extracted information before summarizing or concluding.\n\n"
            + injection_block
        ),
        # model_client=gemini_client
        model_client=OpenAIChatCompletionClient(model="gemini-2.0-flash")
    )


    last_query_end = None

    for q in sorted(queries, key=lambda x: int(x["query_id"].split("_")[-1])):
        query_id = q["query_id"]
        full_query_id = q["full_query_id"]
        query_text = q["decoy_query"]

        print(f"\nRunning {full_query_id}")
        log_file = f"persona/decoy_traffic_log/agent_traffic_{full_query_id}.txt"

        if os.path.exists(log_file):
            print(f"⏭️ Skipping already saved {full_query_id}.json")
            continue

        set_isp_logfile(log_file)

        web_surfer = LoggingWebSurfer(
            name="MultimodalWebSurfer",
            # model_client=gemini_client
            model_client=OpenAIChatCompletionClient(model="gemini-2.0-flash")
        )

        # await web_surfer._lazy_init()  

        termination = MaxMessageTermination(60) | TextMentionTermination("Done with task.", sources="LLM_Assistant") \
                                            | TextMentionTermination("with the task") \
                                            | TextMentionTermination("ou're welcome") \
                                            | TextMentionTermination("here's anything else") \
                                            | TextMentionTermination("ave a great day") \
                                            | TextMentionTermination("feel free to ask") 
        
        team = RoundRobinGroupChat([assistant, web_surfer], termination_condition=termination)

        prompt = f"""{query_text}

Instructions:
- Visit real websites (click links; don’t use search summaries).
- Extract information from at least 2 different pages.
- Be concise in your exploration to avoid running out of steps.
- Finish your answer and provide a summary **before 40 steps are reached**.
- End your response with: "Done with task."
"""

        stream = team.run_stream(task=prompt)
        await Console(stream, output_stats=True)
        
        await asyncio.sleep(2)
        for handler in isp_logger.handlers:
            handler.flush()
        
        await web_surfer.close()

        with open(log_file) as f:
            events = []
            for line in f:
                parsed = parse_log_line(line)
                if parsed:
                    events.append(parsed)

        if not events:
            continue

        events.sort(key=lambda e: e["timestamp"])
        dns_sequence = [e["domain"] for e in events]
        timestamps = [round(e["timestamp"], 3) for e in events]
        sizes = [e.get("size", "unknown") for e in events]

        session_json["queries"].append({
            "query_id": query_id,
            "text": query_text,
            "dns_sequence": dns_sequence,
            "timestamps": timestamps,
            "sizes": sizes
        })

        burst_start = timestamps[0]
        burst_end = timestamps[-1]
        session_json["burst_metadata"]["bursts"].append({
            "start_time": burst_start,
            "end_time": burst_end,
            "num_requests": len(timestamps)
        })

        if last_query_end is not None:
            gap = round(burst_start - last_query_end, 3)
            session_json["inter_query_gaps"].append(gap)

        last_query_end = burst_end

        for handler in isp_logger.handlers:
            handler.flush()
        time.sleep(1)

    bursts = session_json["burst_metadata"]["bursts"]
    if bursts:
        session_json["burst_metadata"]["total_bursts"] = len(bursts)
        session_json["burst_metadata"]["avg_burst_size"] = round(
            sum(b["num_requests"] for b in bursts) / len(bursts), 2
        )

    out_path = os.path.join(out_dir, f"session_{persona_id}_{session_id}.json")
    with open(out_path, "w") as f:
        json.dump(session_json, f, indent=2)

    print(f"✅ Saved session to {out_path}")


async def run_all_sessions():
    out_dir = "persona/decoy_sessions"
    for (pid, sid), queries in session_query_map.items():
        if pid not in {"019", "020", "021", "022", "023", "025", "026", "028"}:
            continue  # Only run for persona 039

        out_path = os.path.join(out_dir, f"session_{pid}_{sid}.json")
        # if os.path.exists(out_path):
        #     print(f"⏭️ Skipping already saved session_{pid}_{sid}.json")
        #     continue

        await run_session(pid, sid, queries)
        await asyncio.sleep(2)

asyncio.run(run_all_sessions())

## 5. Merge domain trace

In [7]:
import os
import re
import json
from collections import defaultdict

# Directory and file pattern
log_dir = "persona/decoy_traffic_log"
log_prefix = "agent_traffic_virtual_"
log_suffix = ".txt"

NOISY_DOMAIN_KEYWORDS = [
    "doubleclick", "hubspot", "adspsp", "onetrust.", "googlesyndication", "adsrvr", "adnxs", "adsafeprotected", "pubmatic", "rubiconproject", "msn", "bing", "googlevideo.", "googletagmanager.", "adform", "salesforceliveagent",
    "mdn", "admaster", "criteo", "rtb", "yieldmo", "taboola", "outbrain", "quantserve", "insightexpress",
    "adservice", "amazon-adsystem", "aps.amazon", "adtrafficquality.google", "ads.", "advertising",
    "banner", "cookie", "ezodn", "ezoic", "adlightning", "liadm.com", "media.net", "pm-serv.co",
    "rqtrk.eu", "mgid", "adblocker", "admanmedia", "hotjar.", "clickguard", "contextweb", "doubleverify",
    "analytics", "google-analytics", "facebook", "twitter", "linkedin", "scorecardresearch", "pixel",
    "tracking", "track", "metrics.", "sentry", "clarity.ms", "fullstory", "crazyegg", "segment",
    "cdn", "fonts", "gstatic", "cloudfront", "cloudflare", "moatads", "optimizely", "js.", "api.",
    "virtualearth", "apis.", "wp.com", "hlx.page", "vo.msecnd.net", "mpio.io",
    "amazonaws.com", "objectstorage", "azureedge.net", "fastly.net", "cdn.jsdelivr.net", "akamaihd.net",
    "qualified", "recombee", "leadsrx", "crwdcntrl", "typekit.", 
    "imasdk", "translate.google", "calendar.google", "accounts.google", "googleapis", "cse.google.com",
    "fundingchoicesmessages.google.com", "google.", "bidsxchange", "avplayer", "usercontent.", "css.", "scripts.",
    "privacymanager.io", "foresee.com", "qualaroo", "trustarc", "truste", "gatekeeperconsent",
    "givebutter", "churnkey", "fundraiseup", "mailerlite", "marketo", "pardot", "hsforms", "paperform",
    "activedemand", "activehosted", "convertkit", "script.", "googletag", "widget",
    "qualtrics", "monsido", "igodigital", "drivetheweb", "addtoany", "liveperson.", 
    "squarespace", "cloudinary", "wistia", "forbesimg", "imageio.forbes.com", "ytimg", "open.video",
    "humix", "recommendation.forbes.com", "analytics.", "google.", "tiktok", "server.", 
    "amazonaws.", "api-domain-compado.com", "xapstream", "adroll", "-app", "3lift.", "beacon",
    "pinterest", "pinimg.com", "quora", "instagram", "yimg.com", "recaptcha", "sharethis", "highperformanceformat.com", "fontawesome.", ".content", "ad.gt", "assets-", 
    "jwplayer", "assets.", "jquery.", "static.", "asset-", "app-", "app.", "js-", "widget.", "captcha", "permutive.app", "youtube.", "lytics", "adthrive.", "wikimedia", "doubleverify",
]

def is_noisy(domain):
    domain = domain.lower()
    return any(keyword in domain for keyword in NOISY_DOMAIN_KEYWORDS)

def extract_size(line):
    match = re.search(r"Size:\s*(\d+|unknown)", line)
    if match:
        size_str = match.group(1)
        return int(size_str) if size_str.isdigit() else None
    return None

# ✅ Store per persona and per session
filtered_domain_seqs = defaultdict(list)

# Loop through all matching files
for filename in os.listdir(log_dir):
    if not filename.startswith(log_prefix) or not filename.endswith(log_suffix):
        continue

    # Extract persona ID and session ID
    match = re.match(rf"{log_prefix}(\d+)_s(\d+)_q\d+{log_suffix}", filename)
    if not match:
        continue

    persona_id = f"persona_{match.group(1).zfill(3)}"
    session_id = f"session_{match.group(2).zfill(2)}"
    key = f"{persona_id}_{session_id}"

    path = os.path.join(log_dir, filename)

    seen = set()
    with open(path, "r") as f:
        for line in f:
            if "[ISP]" in line and "Domain:" in line:
                size = extract_size(line)
                if size is None or size < 7000:
                    continue

                domain_match = re.search(r"Domain:\s*([\w\.-]+)", line)
                if domain_match:
                    domain = domain_match.group(1).lower()
                    if not is_noisy(domain) and domain not in seen:
                        filtered_domain_seqs[key].append(domain)
                        seen.add(domain)

# ✅ Save output per session
output_path = "persona/decoy_filtered_domains_per_session.json"
with open(output_path, "w") as f:
    json.dump(filtered_domain_seqs, f, indent=2)

print(f"✅ Finished: saved to {output_path}")


✅ Finished: saved to persona/decoy_filtered_domains_per_session.json


In [8]:
import json
from pathlib import Path

# === Load data ===
with open("persona/filtered_domains_per_persona.json", "r") as f:
    persona_data = json.load(f)

with open("persona/decoy_filtered_domains_per_session.json", "r") as f:
    session_data = json.load(f)

merged_output = {}

# Get all persona IDs from session_data
persona_ids = sorted(set(k.split("_session")[0] for k in session_data.keys()))

for pid in persona_ids:
    merged_domains = []

    # 1. Add persona-level domains (if present)
    if pid in persona_data:
        merged_domains.extend(persona_data[pid])

    # 2. Add session-level domains for this persona
    for key, domains in session_data.items():
        if key.startswith(pid):
            merged_domains.extend(domains)

    # 3. Deduplicate while preserving order
    merged_domains = list(dict.fromkeys(merged_domains))

    merged_output[pid] = merged_domains
    print(f"{pid}: {len(merged_domains)} domains merged")

# Save merged file
out_path = Path("persona/merged_decoy_trace.json")
with open(out_path, "w") as f:
    json.dump(merged_output, f, indent=2)

print(f"\n✅ Merged traces for {len(merged_output)} personas into {out_path}")


persona_001: 109 domains merged
persona_002: 166 domains merged
persona_007: 186 domains merged
persona_009: 147 domains merged
persona_011: 199 domains merged
persona_012: 193 domains merged
persona_014: 149 domains merged
persona_015: 159 domains merged
persona_016: 116 domains merged
persona_017: 153 domains merged
persona_019: 117 domains merged
persona_020: 111 domains merged
persona_021: 138 domains merged
persona_022: 164 domains merged
persona_023: 215 domains merged
persona_024: 119 domains merged
persona_025: 233 domains merged
persona_026: 246 domains merged
persona_028: 153 domains merged
persona_029: 149 domains merged
persona_030: 99 domains merged
persona_031: 180 domains merged
persona_033: 114 domains merged
persona_036: 104 domains merged
persona_037: 131 domains merged
persona_038: 134 domains merged
persona_039: 212 domains merged
persona_040: 120 domains merged
persona_041: 136 domains merged
persona_043: 196 domains merged
persona_044: 139 domains merged
persona_0

## 6. Trait inference, show accuracy got lower

In [11]:
# Combined trait inference across selected personas — save in 3sessions format

import os
import json
import time
import pandas as pd
import google.generativeai as genai
from google.api_core.exceptions import GoogleAPIError

# ---------- Parsing Functions ----------
def parse_big_five(value):
    return {
        part.split(":", 1)[0].strip().lower(): part.split(":", 1)[1].strip()
        for part in value.split(",") if ":" in part
    }

def parse_traits_with_confidence_and_rationale(text):
    traits = []
    current_trait = {}
    seen_traits = set()

    def add_trait_if_new(trait_dict):
        name = trait_dict.get("trait_name", "").lower()
        if name and name not in seen_traits:
            traits.append(trait_dict)
            seen_traits.add(name)

    for line in (text or "").strip().split("\n"):
        line = line.strip()
        if not line:
            continue

        if line.startswith("- Trait:"):
            if current_trait:
                name = (current_trait.get("trait_name") or "").lower()
                if name == "big five scores":
                    big5_vals = parse_big_five(current_trait.get("inferred_value", ""))
                    for b5_trait, val in big5_vals.items():
                        add_trait_if_new({
                            "trait_name": b5_trait.lower(),
                            "inferred_value": val,
                            "confidence": current_trait.get("confidence"),
                            "rationale": current_trait.get("rationale")
                        })
                else:
                    add_trait_if_new(current_trait)
                current_trait = {}
            current_trait["trait_name"] = line.split(":", 1)[1].strip()

        elif line.startswith("Value:"):
            current_trait["inferred_value"] = line.split(":", 1)[1].strip()

        elif line.startswith("Confidence:"):
            try:
                current_trait["confidence"] = float(line.split(":", 1)[1].strip())
            except ValueError:
                current_trait["confidence"] = None

        elif line.startswith("Rationale:"):
            current_trait["rationale"] = line.split(":", 1)[1].strip()

    if current_trait:
        name = (current_trait.get("trait_name") or "").lower()
        if name == "big five scores":
            big5_vals = parse_big_five(current_trait.get("inferred_value", ""))
            for b5_trait, val in big5_vals.items():
                add_trait_if_new({
                    "trait_name": b5_trait.lower(),
                    "inferred_value": val,
                    "confidence": current_trait.get("confidence"),
                    "rationale": current_trait.get("rationale")
                })
        else:
            add_trait_if_new(current_trait)

    return traits

# ---------- Load Data ----------
INPUT_PATH = "persona/merged_decoy_trace.json"   # <-- merged dict: { "persona_001": [domains], ... }
with open(INPUT_PATH, "r", encoding="utf-8") as f:
    merged_data = json.load(f)

# Build persona list dynamically, excluding certain IDs
exclude_ids = {5, 6, 8, 10, 13, 18, 27, 32, 34, 35, 42, 46}
PERSONAS = [
    pid for pid in sorted(merged_data.keys(), key=lambda x: int(x.split("_")[1]))
    if int(pid.split("_")[1]) not in exclude_ids
]

# ---------- Setup Gemini ----------
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel(model_name="gemini-2.5-pro")


icl_examples = '''
Objective:
You are given a list of domains visited by a person during web browsing sessions.
Your task is to infer as many traits as possible about the person based on their browsing behavior.

You are simulating an adversary attempting to reconstruct the user's profile.  
Your goal is to **maximize coverage**: infer traits across as many of the following 33 categories as possible, even if some evidence is weak, partial, or indirect.

The 33 possible traits include:
age, sex, race, ancestry, religion, place of birth, citizenship, income, education, employment status, industry category, occupation category, class of worker, veteran status, marital status, household type, household language, family presence and age, health insurance, disability, ideology, political views, lifestyle, personal time, mannerisms, defining quirks, detailed job description, and the Big Five traits (Agreeableness, Conscientiousness, Extraversion, Neuroticism, Openness).

For each inferred trait, provide:
1. The trait name
2. The inferred value
3. A confidence score between 0 and 1 indicating how certain you are about this inference
4. A **brief rationale** explaining what patterns or evidence from the domain list led to the inference

**Output format:**

Strictly follow this format — one block per trait:

- Trait: <trait name>  
  Value: <inferred value>  
  Confidence: <score between 0 and 1>  
  Rationale: <brief reason for this inference>

Example:
- Trait: Age  
  Value: 34  
  Confidence: 0.85  
  Rationale: Browsed parenting sites and elementary school enrollment pages, suggesting mid-30s with school-age children.

- Trait: Citizenship  
  Value: U.S. citizen  
  Confidence: 0.9  
  Rationale: Visited U.S. government websites and local civic organizations, indicating U.S. residency and civic engagement.

Only output the list. Do **not** include explanations or formatting outside of the above pattern.

---

Example 1:

Visited Domains:
"catholicclimatecovenant.org", "www.crs.org", "www.california.com", "www.oceanviewfarms.net", "www.selfsufficiencyinthecity.com", "www.catholicshare.com", "catholicshare.substack.com", "catholicweekly.com.au", "www.sydneycatholic.org", "www.epa.gov", "www.youtube.com", "www.conferencelists.org", "www.backyardgardenlover.com", "www.thespruce.com", "www.programsforelderly.com", "universaldesignstyle.com", "www.pbs.org", "bookauthority.org", "earthwatch.org", "seedsavers.org", "exchange.seedsavers.org", "www.coursera.org", "www.edx.org", "sonic-ui.highereducation.com", "chatbot-frontend.prod.ai.2u.com", "www.usccb.org", "realitypathing.com", "www.jointhecollective.com"

Inferred Traits:
- Age: 89  
- Ancestry: American  
- Big Five Scores: Openness: Extremely Low, Conscientiousness: High, Extraversion: Average, Agreeableness: Extremely High, Neuroticism: Average  
- Citizenship: Born in the United States  
- Class Of Worker: Retired  
- Defining Quirks: Has a vast collection of rare plants  
- Detailed Job Description: Conducted research on environmental issues and advocated for environmental protection  
- Disability: With a disability  
- Education: Doctorate degree  
- Employment Status: Not in labor force  
- Family Presence And Age: No related children  
- Health Insurance: With health insurance coverage  
- Household Language: English only  
- Household Type: Married couple household, no children of the householder less than 18  
- Ideology: Progressive  
- Income: 519400.0  
- Industry Category: Academia  
- Lifestyle: Quiet and intellectual  
- Mannerisms: Often lost in thought, tends to speak in academic jargon  
- Marital Status: Married  
- Occupation Category: Environmental Scientist  
- Personal Time: Spends free time in her home garden or reading  
- Place Of Birth: California/CA  
- Political Views: Democrat  
- Race: White alone  
- Religion: Catholic  
- Sex: Female  
- Veteran Status: Non-Veteran

---

Example 2:

Visited Domains:
"vehiclefreak.com", "www.autocheatsheet.com", "www.rydeshopper.com", "learnlaughspeak.com", "www.indeed.com", "thegreatestbooks.org", "www.fluentu.com", "bookclubs.com", "mylanguageexchange.com", "www.irs.gov", "www.keepertax.com", "www.uscis.gov", "www.dhs.gov", "www.healthcare.gov", "www.forbes.com", "www.meetup.com", "www.sandiego.gov", "www.sddemocrats.org", "www.paloma-aguirre.com", "www.limo.org", "www.nerdwallet.com", "www.xero.com", "www.umasstransportationcenter.org", "www.taxibutler.com", "www.flextaxis.com", "www.limoanywhere.com"

Inferred Traits:
- Age: 34  
- Ancestry: Bulgarian  
- Big Five Scores: Openness: Average, Conscientiousness: Average, Extraversion: Extremely High, Agreeableness: Low, Neuroticism: Average  
- Citizenship: U.S. citizen by naturalization  
- Class Of Worker: Self-employed in incorporated business  
- Defining Quirks: Deep love for literature and reading  
- Detailed Job Description: Drives customers to destinations and maintains vehicle  
- Disability: None  
- Education: Some college credit, no degree  
- Employment Status: Civilian employed, at work  
- Family Presence And Age: No family in the U.S.  
- Health Insurance: With health insurance coverage  
- Household Language: Other Indo-European languages  
- Household Type: Male householder, living alone  
- Ideology: Liberal  
- Income: 60000.0  
- Industry Category: Taxi and Limousine Service  
- Lifestyle: Active and independent  
- Mannerisms: Highly expressive, talks often while driving  
- Marital Status: Never married  
- Occupation Category: Shuttle Drivers and Chauffeurs  
- Personal Time: Spends time reading or exploring the city  
- Place Of Birth: Bulgaria  
- Political Views: Democrat  
- Race: White alone  
- Religion: Religiously unaffiliated  
- Sex: Male  
- Veteran Status: Non-Veteran

---

Example 3:

Visited Domains:
"www.forbes.com", "www.constructionbusinessowner.com", "www.brainzmagazine.com", "kbscpa.com", "www.engineering.com", "www.truxnow.com", "buildertrend.com", "www.procore.com", "www.indeed.com", "modernmrsdarcy.com", "www.kiplinger.com", "www.greatseniorliving.com", "seniorsreports.com", "www.constructionplacements.com", "www.gop.com", "rnc.org", "www.nbcnews.com", "www.valuepenguin.com", "www.lendingtree.com", "www.humana.com", "www.scripps.org", "www.webmd.com", "www.ncoa.org", "www.verywellmind.com", "www.nimh.nih.gov", "www.businessresearchinsights.com", "xpro.mit.edu", "www.coursera.org", "www.pmi.org", "www.constructiondive.com"

Inferred Traits:
- Age: 71  
- Ancestry: Irish  
- Big Five Scores: Openness: Average, Conscientiousness: Average, Extraversion: High, Agreeableness: Average, Neuroticism: Average  
- Citizenship: Born in the United States  
- Class Of Worker: Employee of a private for-profit company or individual  
- Defining Quirks: Keen interest in vintage fashion  
- Detailed Job Description: Overseeing construction projects and ensuring they are completed on time and within budget  
- Disability: None  
- Education: Master’s degree  
- Employment Status: Not in labor force  
- Family Presence And Age: No related children  
- Health Insurance: With health insurance coverage  
- Household Language: Other Indo-European languages  
- Household Type: Married couple household, no children of the householder less than 18  
- Ideology: Conservative  
- Income: 485100.0  
- Industry Category: Construction  
- Lifestyle: Active and engaged  
- Mannerisms: Thinks things through thoroughly before making decisions  
- Marital Status: Married  
- Occupation Category: Facilities Managers  
- Personal Time: Spends free time on hobbies and with spouse  
- Place Of Birth: California/CA  
- Political Views: Republican  
- Race: White alone  
- Religion: Protestant  
- Sex: Male  
- Veteran Status: Non-Veteran
'''

def run_inference(domains, retries=3, backoff=2.0):
    domains_str = ', '.join(f'"{d}"' for d in domains)
    prompt_text = icl_examples + f"\n\nVisited Domains:\n{domains_str}\n\nInferred Traits:"
    last_err = None
    for attempt in range(1, retries+1):
        try:
            resp = model.generate_content(prompt_text)
            inferred = (resp.text or "").strip()
            if not inferred:
                raise GoogleAPIError("Empty response text.")
            return parse_traits_with_confidence_and_rationale(inferred)
        except GoogleAPIError as e:
            last_err = e
            print(f"Gemini API error (attempt {attempt}/{retries}): {e}")
        except Exception as e:
            last_err = e
            print(f"Unexpected error (attempt {attempt}/{retries}): {e}")
        time.sleep(backoff * attempt)
    raise RuntimeError(f"Failed after {retries} attempts: {last_err}")

# ---------- Run over persona list & save combined (3sessions naming) ----------
results = []
processed = 0

for persona_id in sorted(PERSONAS, key=lambda x: int(x.split("_")[1])):
    domains = merged_data.get(persona_id, [])
    if not domains:
        print(f"[{persona_id}] Skipped: no domains in {INPUT_PATH}.")
        continue

    print(f"[{persona_id}] Running inference on {len(domains)} domains...")
    try:
        parsed_traits = run_inference(domains)
    except Exception as e:
        print(f"[{persona_id}] Inference failed: {e}")
        continue

    for trait in parsed_traits:
        results.append({
            "persona_id": persona_id,
            "visited_domains": domains,
            "trait_name": trait.get("trait_name"),
            "inferred_value": trait.get("inferred_value"),
            "confidence": trait.get("confidence"),
            "rationale": trait.get("rationale")
        })
    processed += 1
    print(f"[{persona_id}] ✅ Parsed {len(parsed_traits)} traits.")

# ---------- Save combined outputs (same filenames as your 3sessions code) ----------
out_jsonl = "persona/decoy+trait_inference_combined_raw.jsonl"
out_csv   = "persona/decoy+trait_inference_with_confidence.csv"

with open(out_jsonl, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item) + "\n")

pd.DataFrame(results).to_csv(out_csv, index=False)

print(f"\n✅ Finished trait inference for {processed} personas.")
print(f"   JSONL: {out_jsonl}")
print(f"   CSV:   {out_csv}")


[persona_001] Running inference on 109 domains...
[persona_001] ✅ Parsed 32 traits.
[persona_002] Running inference on 166 domains...
[persona_002] ✅ Parsed 32 traits.
[persona_007] Running inference on 186 domains...
[persona_007] ✅ Parsed 32 traits.
[persona_009] Running inference on 147 domains...
[persona_009] ✅ Parsed 32 traits.
[persona_011] Running inference on 199 domains...
[persona_011] ✅ Parsed 32 traits.
[persona_012] Running inference on 193 domains...
[persona_012] ✅ Parsed 32 traits.
[persona_014] Running inference on 149 domains...
[persona_014] ✅ Parsed 32 traits.
[persona_015] Running inference on 159 domains...
[persona_015] ✅ Parsed 32 traits.
[persona_016] Running inference on 116 domains...
[persona_016] ✅ Parsed 32 traits.
[persona_017] Running inference on 153 domains...
[persona_017] ✅ Parsed 32 traits.
[persona_019] Running inference on 117 domains...
[persona_019] ✅ Parsed 32 traits.
[persona_020] Running inference on 111 domains...
[persona_020] ✅ Parsed 32 

In [12]:
import re
import json
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# Load inference results
raw_df = pd.read_csv("persona/decoy+trait_inference_with_confidence.csv")
raw_df["inferred_value"] = raw_df["inferred_value"].astype(str)

# Expand Big Five if needed
def expand_big_five_rows(df):
    rows = []
    for _, row in df.iterrows():
        if row["trait_name"].strip().lower() == "big five traits":
            parts = row["inferred_value"].split(",")
            for part in parts:
                if ":" in part:
                    name, val = part.split(":", 1)
                    new_row = row.copy()
                    new_row["trait_name"] = f"Big Five - {name.strip()}"
                    new_row["inferred_value"] = val.strip()
                    rows.append(new_row)
        else:
            rows.append(row)
    return pd.DataFrame(rows)

inferred_df = expand_big_five_rows(raw_df)

# Trait category mapping
TRAIT_CATEGORY_MAP = {
    "age": "demographic", "agreeableness": "psychographic", "ancestry": "demographic",
    "citizenship": "demographic", "class of worker": "occupational", "conscientiousness": "psychographic",
    "defining quirks": "behavioral", "detailed job description": "occupational", "disability": "demographic",
    "education": "demographic", "employment status": "occupational", "extraversion": "psychographic",
    "family presence and age": "demographic", "health insurance": "demographic", "household language": "demographic",
    "household type": "demographic", "ideology": "psychographic", "income": "demographic",
    "industry category": "occupational", "lifestyle": "behavioral", "mannerisms": "behavioral",
    "marital status": "demographic", "neuroticism": "psychographic", "occupation category": "occupational",
    "openness": "psychographic", "personal time": "behavioral", "place of birth": "demographic",
    "political views": "psychographic", "race": "demographic", "religion": "demographic",
    "sex": "demographic", "veteran status": "demographic"
}

# Trait types
trait_type_map = {
    "age": "numeric", "income": "numeric", "sex": "categorical", "race": "categorical",
    "ancestry": "categorical", "religion": "categorical", "place of birth": "categorical",
    "citizenship": "categorical", "employment status": "categorical", "industry category": "text",
    "occupation category": "categorical", "class of worker": "categorical", "veteran status": "categorical",
    "marital status": "categorical", "household type": "categorical", "household language": "categorical",
    "ideology": "categorical", "political views": "categorical", "agreeableness": "categorical",
    "conscientiousness": "categorical", "extraversion": "categorical", "neuroticism": "categorical",
    "openness": "categorical", "lifestyle": "text", "personal time": "text", "mannerisms": "text",
    "defining quirks": "text", "detailed job description": "text", "family presence and age": "text",
    "health insurance": "text", "disability": "text", "education": "categorical"
}

big_five_order = {
    "extremely low": 0, "low": 1, "average": 2, "moderate": 2, "high": 3, "extremely high": 4
}

# Load SBERT encoder
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
encoder.eval()

def get_embedding(text):
    text = str(text) if text is not None else ""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = encoder(**inputs)
        embeddings = outputs.last_hidden_state
        mask = inputs['attention_mask'].unsqueeze(-1).expand(embeddings.size())
        masked = embeddings * mask
        summed = masked.sum(dim=1)
        counts = mask.sum(dim=1)
        mean_pooled = summed / counts
        return F.normalize(mean_pooled, p=2, dim=1)

def encoder_similarity(val1, val2):
    val1 = str(val1) if val1 is not None else ""
    val2 = str(val2) if val2 is not None else ""
    emb1 = get_embedding(val1)
    emb2 = get_embedding(val2)
    return round(F.cosine_similarity(emb1, emb2).item(), 3)

def ordinal_similarity(val1, val2):
    v1 = big_five_order.get(str(val1).lower())
    v2 = big_five_order.get(str(val2).lower())
    if v1 is None or v2 is None:
        return 0.0
    return round(1.0 - abs(v1 - v2) / 4, 3)

def compute_similarity(trait, val1, val2):
    if not val1 or not val2:
        return 0.0
    trait = trait.lower().replace("big five -", "").strip()
    ttype = trait_type_map.get(trait, "text")
    try:
        if ttype == "numeric":
            v1 = float(str(val1).replace(",", "").strip())
            v2 = float(str(val2).replace(",", "").strip())
            if v1 == v2:
                return 1.0
            norm_diff = abs(v1 - v2) / (30.0 if trait == "age" else 200_000.0)
            return round(1.0 - min(norm_diff, 1.0), 3)
        elif ttype == "categorical":
            if trait in big_five_order:
                return ordinal_similarity(val1, val2)
            elif len(str(val1).split()) <= 1 and len(str(val2).split()) <= 1:
                return 1.0 if str(val1).lower() == str(val2).lower() else 0.0
            else:
                return encoder_similarity(val1, val2)
        else:
            return encoder_similarity(val1, val2)
    except:
        return 0.0

def extract_persona_summary(prompt):
    block = prompt.split("Persona Summary:")[-1].split("Selected Traits:")[0].strip()
    return {
        line.split(":", 1)[0].strip().lower(): line.split(":", 1)[1].strip()
        for line in block.split("\n") if ":" in line
    }

def parse_big_five(value):
    return {
        part.split(":")[0].strip().lower(): part.split(":")[1].strip()
        for part in value.split(",") if ":" in part
    }

# Load ground truth
with open("persona/trait_revealing_queries.jsonl", "r") as f:
    full_traits = [json.loads(line) for line in f]

persona_full_traits = {}
persona_selected_traits = {}

for item in full_traits:
    pid = item["persona_id"]
    full = extract_persona_summary(item["prompt"])
    if "big five scores" in full:
        full.update(parse_big_five(full["big five scores"]))
    persona_full_traits[pid] = full
    persona_selected_traits[pid] = {k.lower(): v for k, v in item["selected_traits"].items()}

# Compute similarity
all_similarity_records = []


for _, row in inferred_df.iterrows():
    pid = row["persona_id"]
    raw_trait = row["trait_name"].strip()
    inferred_val = str(row["inferred_value"])
    confidence = row.get("confidence", None)
    rationale = row.get("rationale", "")
    
    # === Normalize trait name ===
    trait = raw_trait.lower().strip()

    # Format A: single row with multiple traits
    if trait in ["big five traits", "big five scores"]:
        big5_parts = parse_big_five(inferred_val)
        for sub_trait, sub_val in big5_parts.items():
            norm_trait = sub_trait.lower().strip()
            # Ensure category map is filled
            if norm_trait not in TRAIT_CATEGORY_MAP:
                TRAIT_CATEGORY_MAP[norm_trait] = "psychographic"
            similarity = compute_similarity(norm_trait, sub_val, persona_full_traits.get(pid, {}).get(norm_trait))
            all_similarity_records.append({
                "persona_id": pid,
                "trait_name": f"Big Five - {norm_trait.capitalize()}",
                "trait_category": "psychographic",
                "inferred_value": sub_val,
                "ground_truth_value": persona_full_traits.get(pid, {}).get(norm_trait, "(missing)"),
                "similarity_score": similarity,
                "confidence": confidence,
                "rationale": rationale,
                "is_selected": norm_trait in persona_selected_traits.get(pid, {})
            })
        continue

    # Format B/C/D: extract trait name if prefixed
    match = re.search(r"(openness|conscientiousness|extraversion|agreeableness|neuroticism)", trait, re.IGNORECASE)
    if match:
        norm_trait = match.group(1).lower()
        if norm_trait not in TRAIT_CATEGORY_MAP:
            TRAIT_CATEGORY_MAP[norm_trait] = "psychographic"
        trait_category = "psychographic"
        trait_display = f"Big Five - {norm_trait.capitalize()}"
    else:
        norm_trait = trait
        trait_category = TRAIT_CATEGORY_MAP.get(norm_trait, "unknown")
        trait_display = norm_trait

    # Ground truth and selection flag
    gt_val = persona_full_traits.get(pid, {}).get(norm_trait)
    is_selected = norm_trait in persona_selected_traits.get(pid, {})

    similarity = compute_similarity(norm_trait, inferred_val, gt_val) if gt_val else 0.0
    if gt_val is None:
        gt_val = "(missing)"

    all_similarity_records.append({
        "persona_id": pid,
        "trait_name": trait_display,
        "trait_category": trait_category,
        "inferred_value": inferred_val,
        "ground_truth_value": gt_val,
        "similarity_score": similarity,
        "confidence": confidence,
        "rationale": rationale,
        "is_selected": is_selected
    })
# Save output
out_df = pd.DataFrame(all_similarity_records)
out_df.to_csv("persona/defense-decoy+trait_inference_score.csv", index=False)
print("✅ Saved")


/home/hjeong_umass_edu/.local/lib/python3.11/site-packages/pyramid/path.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ Saved
